### Imports and helper functions

In [ ]:
# %run helper.py
%run ./library/distributions_and_points_visualizations.py

In [ ]:
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from itertools import cycle
from scipy.optimize import linear_sum_assignment
from IPython.display import display, HTML

In [ ]:
def visualize_points(pts):
    plt.plot(pts[:, 0], pts[:, 1], '.', alpha=0.5)
    plt.axis('equal')
    plt.grid()

    return plt

In [ ]:
def create_ellipse_points(mean, cov, n_std=1.96, n_points=100):
    eigenvalues, eigenvectors = np.linalg.eig(cov)
    angle = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])
    
    t = np.linspace(0, 2*np.pi, n_points)
    ellipse_x = n_std * np.sqrt(eigenvalues[0]) * np.cos(t)
    ellipse_y = n_std * np.sqrt(eigenvalues[1]) * np.sin(t)
    
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle), np.cos(angle)]])
    
    ellipse_points = np.dot(np.column_stack([ellipse_x, ellipse_y]), R.T) + mean
    return ellipse_points[:, 0], ellipse_points[:, 1]

def plot_confidence_ellipses(distributions, n_std=1.96):
    fig, ax = plt.subplots()

    cmap = plt.get_cmap('viridis')
    colors = cycle(cmap(np.linspace(0, 1, len(distributions))))
    patches = []

    for idx, dist in enumerate(distributions):
        mean, cov = dist[0], np.array([dist[1], dist[2]])
        x, y = create_ellipse_points(mean, cov, n_std)
        
        color = next(colors)
        
        # Create polygon
        polygon_points = np.column_stack([x, y])
        poly = Polygon(polygon_points, closed=True, fill=True, color=color, alpha=0.5)
        ax.add_patch(poly)
        patches.append(poly)

    ax.set_title('Confidence Ellipses')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.legend(patches, ['Distribution {}'.format(i) for i in range(len(distributions))])
    ax.axis('equal')

    return plt, ax

def plot_distributions_with_points(distributions, points, n_std=1.96):
    fig, ax = plot_confidence_ellipses(distributions, n_std)
    
    ax.plot(points[:, 0], points[:, 1], 'ko')

    return plt

def plot_distributions_with_colored_points(distributions, points, points_to_distribution, n_std=1.96):
    fig, ax = plot_confidence_ellipses(distributions, n_std)
    
    # Plot all points in a default color (black)
    ax.plot(points[:, 0], points[:, 1], 'ko', label='All Points')

    # Use the same colormap to get consistent colors
    cmap = plt.get_cmap('viridis')
    colors = cmap(np.linspace(0, 1, len(distributions)))

    for dist_idx, point_idx in enumerate(points_to_distribution):
        ax.plot(points[point_idx, 0], points[point_idx, 1], 'o', color=colors[dist_idx], label=f'Point to Distribution {dist_idx}')
    
    ax.legend()
    return plt, ax

In [ ]:
assignment_problem_html = """
<!DOCTYPE html> <html><!--
 Page saved with SingleFile 
 url: http://localhost:3001/001-assignment-problem.html 
 saved date: Mon Sep 30 2024 23:37:54 GMT+0200 (Central European Summer Time)
--><meta charset=utf-8>
<meta name=viewport content="width=device-width initial-scale=1">
<style>html{overflow-x:initial!important}:root{--bg-color:#ffffff;--text-color:#333333;--select-text-bg-color:#B5D6FC;--select-text-font-color:auto;--monospace:"Lucida Console",Consolas,"Courier",monospace;--title-bar-height:20px}html{font-size:14px;background-color:var(--bg-color);color:var(--text-color);font-family:"Helvetica Neue",Helvetica,Arial,sans-serif;-webkit-font-smoothing:antialiased}h4,h5{white-space:pre-wrap}body{margin:0px;padding:0px;height:auto;inset:0px;font-size:1rem;line-height:1.428571;overflow-x:hidden;background:inherit}.in-text-selection,::selection{text-shadow:none;background:var(--select-text-bg-color);color:var(--select-text-font-color)}#write{height:auto;width:inherit;word-break:normal;overflow-wrap:break-word;position:relative;white-space:normal;overflow-x:visible;padding-top:36px}body.typora-export{padding-left:30px;padding-right:30px}.typora-export p{white-space:pre-wrap}@media screen and (max-width:500px){body.typora-export{padding-left:0px;padding-right:0px}#write{padding-left:20px;padding-right:20px}}img{vertical-align:middle;image-orientation:from-image}*,::after,::before{box-sizing:border-box}#write h4,#write h5,#write p{width:inherit}#write h4,#write h5,#write p{position:relative}p{line-height:inherit}h4,h5{break-after:avoid-page;break-inside:avoid;orphans:4}p{orphans:4}svg{break-inside:avoid}p>.md-image:only-child:not(.md-img-error) img,p>img:only-child{display:block;margin:auto}.md-math-block{width:100%}.md-math-block:not(:empty)::after{display:none}[contenteditable="true"]:active,[contenteditable="true"]:focus,[contenteditable="false"]:active,[contenteditable="false"]:focus{outline:0px;box-shadow:none}@media screen and (max-width:48em){}@media screen and (max-width:1024px){}@media screen and (max-width:800px){}mjx-container{break-inside:avoid}:root{--text-color:#1f2329;--bg-color:white;--side-bar-bg-color:white;--active-file-bg-color:white;--rawblock-edit-panel-bd:#f5f6f7;--window-border:0 solid white;--control-text-color:#1f2329;--primary-color:#3370ff;--primary-btn-border-color:#3370ff;--active-file-border-color:#3370ff;--primary-btn-text-color:#3370ff;--item-hover-text-color:#3370ff;--meta-content-color:#3370ff;--search-select-text-color:#3370ff;--heading-char-color:#3370ff;--mermaid-theme:default}#write{line-height:1.68;-webkit-text-size-adjust:100%;max-width:960px;margin:0px auto;padding:1.6rem 3.2rem;font-family:-apple-system,BlinkMacSystemFont,Helvetica Neue,Arial,Segoe UI,PingFang SC,Microsoft Yahei,Hiragino Sans GB,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-size:16px;color:#1f2329}#write *{-moz-box-sizing:border-box;-webkit-box-sizing:border-box;box-sizing:border-box}#write>*:first-child{margin-top:0!important}#write>*:last-child{margin-top:0!important}#write p{margin-top:8px;margin-bottom:8px}#write h4{font-size:18px;margin-left:0;margin-right:0}#write h5{font-size:16px;margin-left:0;margin-right:0}#write p{margin-left:0;margin-right:0}#write img{max-width:100%;height:auto}#write .md-rawblock{margin-top:1em;margin-bottom:1em}#write *{-webkit-font-smoothing:antialiased;text-rendering:optimizeLegibility}#write h4,#write h5{line-height:1.4;font-weight:500;margin-top:1em;margin-bottom:0.37em}mjx-container[jax="SVG"]{direction:ltr}mjx-container[jax="SVG"]>svg{overflow:visible;min-height:1px;min-width:1px}mjx-assistive-mml{position:absolute!important;top:0px;left:0px;clip:rect(1px,1px,1px,1px);padding:1px 0px 0px 0px!important;border:0px!important;display:block!important;width:auto!important;overflow:hidden!important;-webkit-touch-callout:none;-webkit-user-select:none;-moz-user-select:none;-ms-user-select:none;user-select:none}mjx-assistive-mml[display="block"]{width:100%!important}mjx-container[jax="SVG"][display="true"]{text-align:center;margin:1em 0}mjx-container[jax="SVG"][display="true"][width="full"]{display:flex}g[data-mml-node="mtable"]>g>svg{overflow:visible}mjx-container[jax="SVG"] use[data-c]{stroke-width:0}</style><title>001-assignment-problem</title>
<meta name=referrer content=no-referrer><link rel=canonical href=http://localhost:3001/001-assignment-problem.html><meta http-equiv=content-security-policy content="default-src 'none'; font-src 'self' data:; img-src 'self' data:; style-src 'unsafe-inline'; media-src 'self' data:; script-src 'unsafe-inline' data:; object-src 'self' data:; frame-src 'self' data:;"><style>img[src="data:,"],source[src="data:,"]{display:none!important}</style></head>
<body class=typora-export><div class=typora-export-content>
<div id=write><h4 id=problem><span>Problem</span></h4><p><span>For each point we can calculate the likelihood, that this point belongs to one of the distributions. How can we determine what the most-likely mappig from point-to-distribution is?</span><p><img src="..." width=550><h5 id=the-assignment-problem><span>The "Assignment Problem"</span></h5><p><span>Let us define a set </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=1.699ex height=1.545ex role=img focusable=false viewBox="0 -683 751 683" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:0px><defs><path id=MJX-1639-TEX-I-1D443 d="M287 628Q287 635 230 637Q206 637 199 638T192 648Q192 649 194 659Q200 679 203 681T397 683Q587 682 600 680Q664 669 707 631T751 530Q751 453 685 389Q616 321 507 303Q500 302 402 301H307L277 182Q247 66 247 59Q247 55 248 54T255 50T272 48T305 46H336Q342 37 342 35Q342 19 335 5Q330 0 319 0Q316 0 282 1T182 2Q120 2 87 2T51 1Q33 1 33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM645 554Q645 567 643 575T634 597T609 619T560 635Q553 636 480 637Q463 637 445 637T416 636T404 636Q391 635 386 627Q384 621 367 550T332 412T314 344Q314 342 395 342H407H430Q542 342 590 392Q617 419 631 471T645 554Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D443 xlink:href=#MJX-1639-TEX-I-1D443></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>P</mi></math></mjx-assistive-mml></mjx-container><span> that consists of all points that were in the top </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=1.061ex height=1.023ex role=img focusable=false viewBox="0 -442 469 452" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.023ex><defs><path id=MJX-1511-TEX-I-1D460 d="M131 289Q131 321 147 354T203 415T300 442Q362 442 390 415T419 355Q419 323 402 308T364 292Q351 292 340 300T328 326Q328 342 337 354T354 372T367 378Q368 378 368 379Q368 382 361 388T336 399T297 405Q249 405 227 379T204 326Q204 301 223 291T278 274T330 259Q396 230 396 163Q396 135 385 107T352 51T289 7T195 -10Q118 -10 86 19T53 87Q53 126 74 143T118 160Q133 160 146 151T160 120Q160 94 142 76T111 58Q109 57 108 57T107 55Q108 52 115 47T146 34T201 27Q237 27 263 38T301 66T318 97T323 122Q323 150 302 164T254 181T195 196T148 231Q131 256 131 289Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D460 xlink:href=#MJX-1511-TEX-I-1D460></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>s</mi></math></mjx-assistive-mml></mjx-container><span> likelihood values for any of the distributions. We can now construct a matrix </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=2.378ex height=1.545ex role=img focusable=false viewBox="0 -683 1051 683" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:0px><defs><path id=MJX-1513-TEX-I-1D440 d="M289 629Q289 635 232 637Q208 637 201 638T194 648Q194 649 196 659Q197 662 198 666T199 671T201 676T203 679T207 681T212 683T220 683T232 684Q238 684 262 684T307 683Q386 683 398 683T414 678Q415 674 451 396L487 117L510 154Q534 190 574 254T662 394Q837 673 839 675Q840 676 842 678T846 681L852 683H948Q965 683 988 683T1017 684Q1051 684 1051 673Q1051 668 1048 656T1045 643Q1041 637 1008 637Q968 636 957 634T939 623Q936 618 867 340T797 59Q797 55 798 54T805 50T822 48T855 46H886Q892 37 892 35Q892 19 885 5Q880 0 869 0Q864 0 828 1T736 2Q675 2 644 2T609 1Q592 1 592 11Q592 13 594 25Q598 41 602 43T625 46Q652 46 685 49Q699 52 704 61Q706 65 742 207T813 490T848 631L654 322Q458 10 453 5Q451 4 449 3Q444 0 433 0Q418 0 415 7Q413 11 374 317L335 624L267 354Q200 88 200 79Q206 46 272 46H282Q288 41 289 37T286 19Q282 3 278 1Q274 0 267 0Q265 0 255 0T221 1T157 2Q127 2 95 1T58 0Q43 0 39 2T35 11Q35 13 38 25T43 40Q45 46 65 46Q135 46 154 86Q158 92 223 354T289 629Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D440 xlink:href=#MJX-1513-TEX-I-1D440></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>M</mi></math></mjx-assistive-mml></mjx-container><span> (</span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=11.721ex height=2.111ex role=img focusable=false viewBox="0 -893 5180.8 933" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.09ex><defs><path id=MJX-1766-TEX-I-1D440 d="M289 629Q289 635 232 637Q208 637 201 638T194 648Q194 649 196 659Q197 662 198 666T199 671T201 676T203 679T207 681T212 683T220 683T232 684Q238 684 262 684T307 683Q386 683 398 683T414 678Q415 674 451 396L487 117L510 154Q534 190 574 254T662 394Q837 673 839 675Q840 676 842 678T846 681L852 683H948Q965 683 988 683T1017 684Q1051 684 1051 673Q1051 668 1048 656T1045 643Q1041 637 1008 637Q968 636 957 634T939 623Q936 618 867 340T797 59Q797 55 798 54T805 50T822 48T855 46H886Q892 37 892 35Q892 19 885 5Q880 0 869 0Q864 0 828 1T736 2Q675 2 644 2T609 1Q592 1 592 11Q592 13 594 25Q598 41 602 43T625 46Q652 46 685 49Q699 52 704 61Q706 65 742 207T813 490T848 631L654 322Q458 10 453 5Q451 4 449 3Q444 0 433 0Q418 0 415 7Q413 11 374 317L335 624L267 354Q200 88 200 79Q206 46 272 46H282Q288 41 289 37T286 19Q282 3 278 1Q274 0 267 0Q265 0 255 0T221 1T157 2Q127 2 95 1T58 0Q43 0 39 2T35 11Q35 13 38 25T43 40Q45 46 65 46Q135 46 154 86Q158 92 223 354T289 629Z"></path><path id=MJX-1766-TEX-N-2208 d="M84 250Q84 372 166 450T360 539Q361 539 377 539T419 540T469 540H568Q583 532 583 520Q583 511 570 501L466 500Q355 499 329 494Q280 482 242 458T183 409T147 354T129 306T124 272V270H568Q583 262 583 250T568 230H124V228Q124 207 134 177T167 112T231 48T328 7Q355 1 466 0H570Q583 -10 583 -20Q583 -32 568 -40H471Q464 -40 446 -40T417 -41Q262 -41 172 45Q84 127 84 250Z"></path><path id=MJX-1766-TEX-D-211D d="M17 665Q17 672 28 683H221Q415 681 439 677Q461 673 481 667T516 654T544 639T566 623T584 607T597 592T607 578T614 565T618 554L621 548Q626 530 626 497Q626 447 613 419Q578 348 473 326L455 321Q462 310 473 292T517 226T578 141T637 72T686 35Q705 30 705 16Q705 7 693 -1H510Q503 6 404 159L306 310H268V183Q270 67 271 59Q274 42 291 38Q295 37 319 35Q344 35 353 28Q362 17 353 3L346 -1H28Q16 5 16 16Q16 35 55 35Q96 38 101 52Q106 60 106 341T101 632Q95 645 55 648Q17 648 17 665ZM241 35Q238 42 237 45T235 78T233 163T233 337V621L237 635L244 648H133Q136 641 137 638T139 603T141 517T141 341Q141 131 140 89T134 37Q133 36 133 35H241ZM457 496Q457 540 449 570T425 615T400 634T377 643Q374 643 339 648Q300 648 281 635Q271 628 270 610T268 481V346H284Q327 346 375 352Q421 364 439 392T457 496ZM492 537T492 496T488 427T478 389T469 371T464 361Q464 360 465 360Q469 360 497 370Q593 400 593 495Q593 592 477 630L457 637L461 626Q474 611 488 561Q492 537 492 496ZM464 243Q411 317 410 317Q404 317 401 315Q384 315 370 312H346L526 35H619L606 50Q553 109 464 243Z"></path><path id=MJX-1766-TEX-N-7C d="M139 -249H137Q125 -249 119 -235V251L120 737Q130 750 139 750Q152 750 159 735V-235Q151 -249 141 -249H139Z"></path><path id=MJX-1766-TEX-I-1D443 d="M287 628Q287 635 230 637Q206 637 199 638T192 648Q192 649 194 659Q200 679 203 681T397 683Q587 682 600 680Q664 669 707 631T751 530Q751 453 685 389Q616 321 507 303Q500 302 402 301H307L277 182Q247 66 247 59Q247 55 248 54T255 50T272 48T305 46H336Q342 37 342 35Q342 19 335 5Q330 0 319 0Q316 0 282 1T182 2Q120 2 87 2T51 1Q33 1 33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM645 554Q645 567 643 575T634 597T609 619T560 635Q553 636 480 637Q463 637 445 637T416 636T404 636Q391 635 386 627Q384 621 367 550T332 412T314 344Q314 342 395 342H407H430Q542 342 590 392Q617 419 631 471T645 554Z"></path><path id=MJX-1766-TEX-N-D7 d="M630 29Q630 9 609 9Q604 9 587 25T493 118L389 222L284 117Q178 13 175 11Q171 9 168 9Q160 9 154 15T147 29Q147 36 161 51T255 146L359 250L255 354Q174 435 161 449T147 471Q147 480 153 485T168 490Q173 490 175 489Q178 487 284 383L389 278L493 382Q570 459 587 475T609 491Q630 491 630 471Q630 464 620 453T522 355L418 250L522 145Q606 61 618 48T630 29Z"></path><path id=MJX-1766-TEX-I-1D441 d="M234 637Q231 637 226 637Q201 637 196 638T191 649Q191 676 202 682Q204 683 299 683Q376 683 387 683T401 677Q612 181 616 168L670 381Q723 592 723 606Q723 633 659 637Q635 637 635 648Q635 650 637 660Q641 676 643 679T653 683Q656 683 684 682T767 680Q817 680 843 681T873 682Q888 682 888 672Q888 650 880 642Q878 637 858 637Q787 633 769 597L620 7Q618 0 599 0Q585 0 582 2Q579 5 453 305L326 604L261 344Q196 88 196 79Q201 46 268 46H278Q284 41 284 38T282 19Q278 6 272 0H259Q228 2 151 2Q123 2 100 2T63 2T46 1Q31 1 31 10Q31 14 34 26T39 40Q41 46 62 46Q130 49 150 85Q154 91 221 362L289 634Q287 635 234 637Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D440 xlink:href=#MJX-1766-TEX-I-1D440></use></g><g data-mml-node=mo transform=translate(1328.8,0)><use data-c=2208 xlink:href=#MJX-1766-TEX-N-2208></use></g><g data-mml-node=msup transform=translate(2273.6,0)><g data-mml-node=TeXAtom data-mjx-texclass=ORD><g data-mml-node=mi><use data-c=211D xlink:href=#MJX-1766-TEX-D-211D></use></g></g><g data-mml-node=TeXAtom transform="translate(755,363) scale(0.707)" data-mjx-texclass=ORD><g data-mml-node=mo transform="translate(0 -0.5)"><use data-c=7C xlink:href=#MJX-1766-TEX-N-7C></use></g><g data-mml-node=mi transform=translate(278,0)><use data-c=1D443 xlink:href=#MJX-1766-TEX-I-1D443></use></g><g data-mml-node=mo transform="translate(1029,0) translate(0 -0.5)"><use data-c=7C xlink:href=#MJX-1766-TEX-N-7C></use></g><g data-mml-node=mo transform=translate(1307,0)><use data-c=D7 xlink:href=#MJX-1766-TEX-N-D7></use></g><g data-mml-node=mi transform=translate(2085,0)><use data-c=1D441 xlink:href=#MJX-1766-TEX-I-1D441></use></g></g></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>M</mi><mo>∈</mo><msup><mrow data-mjx-texclass=ORD><mi mathvariant=double-struck>R</mi></mrow><mrow data-mjx-texclass=ORD><mo data-mjx-texclass=ORD stretchy=false>|</mo><mi>P</mi><mo data-mjx-texclass=ORD stretchy=false>|</mo><mo>×</mo><mi>N</mi></mrow></msup></math></mjx-assistive-mml></mjx-container><span>) that contains, for each of the </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=1.699ex height=1.545ex role=img focusable=false viewBox="0 -683 751 683" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:0px><defs><path id=MJX-1639-TEX-I-1D443 d="M287 628Q287 635 230 637Q206 637 199 638T192 648Q192 649 194 659Q200 679 203 681T397 683Q587 682 600 680Q664 669 707 631T751 530Q751 453 685 389Q616 321 507 303Q500 302 402 301H307L277 182Q247 66 247 59Q247 55 248 54T255 50T272 48T305 46H336Q342 37 342 35Q342 19 335 5Q330 0 319 0Q316 0 282 1T182 2Q120 2 87 2T51 1Q33 1 33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM645 554Q645 567 643 575T634 597T609 619T560 635Q553 636 480 637Q463 637 445 637T416 636T404 636Q391 635 386 627Q384 621 367 550T332 412T314 344Q314 342 395 342H407H430Q542 342 590 392Q617 419 631 471T645 554Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D443 xlink:href=#MJX-1639-TEX-I-1D443></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>P</mi></math></mjx-assistive-mml></mjx-container><span> points, the likelihood that it falls within any of the distributions.</span><p><span>For </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=6.343ex height=1.984ex role=img focusable=false viewBox="0 -683 2803.5 877" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.439ex><defs><path id=MJX-1754-TEX-I-1D45D d="M23 287Q24 290 25 295T30 317T40 348T55 381T75 411T101 433T134 442Q209 442 230 378L240 387Q302 442 358 442Q423 442 460 395T497 281Q497 173 421 82T249 -10Q227 -10 210 -4Q199 1 187 11T168 28L161 36Q160 35 139 -51T118 -138Q118 -144 126 -145T163 -148H188Q194 -155 194 -157T191 -175Q188 -187 185 -190T172 -194Q170 -194 161 -194T127 -193T65 -192Q-5 -192 -24 -194H-32Q-39 -187 -39 -183Q-37 -156 -26 -148H-6Q28 -147 33 -136Q36 -130 94 103T155 350Q156 355 156 364Q156 405 131 405Q109 405 94 377T71 316T59 280Q57 278 43 278H29Q23 284 23 287ZM178 102Q200 26 252 26Q282 26 310 49T356 107Q374 141 392 215T411 325V331Q411 405 350 405Q339 405 328 402T306 393T286 380T269 365T254 350T243 336T235 326L232 322Q232 321 229 308T218 264T204 212Q178 106 178 102Z"></path><path id=MJX-1754-TEX-I-1D456 d="M184 600Q184 624 203 642T247 661Q265 661 277 649T290 619Q290 596 270 577T226 557Q211 557 198 567T184 600ZM21 287Q21 295 30 318T54 369T98 420T158 442Q197 442 223 419T250 357Q250 340 236 301T196 196T154 83Q149 61 149 51Q149 26 166 26Q175 26 185 29T208 43T235 78T260 137Q263 149 265 151T282 153Q302 153 302 143Q302 135 293 112T268 61T223 11T161 -11Q129 -11 102 10T74 74Q74 91 79 106T122 220Q160 321 166 341T173 380Q173 404 156 404H154Q124 404 99 371T61 287Q60 286 59 284T58 281T56 279T53 278T49 278T41 278H27Q21 284 21 287Z"></path><path id=MJX-1754-TEX-N-2208 d="M84 250Q84 372 166 450T360 539Q361 539 377 539T419 540T469 540H568Q583 532 583 520Q583 511 570 501L466 500Q355 499 329 494Q280 482 242 458T183 409T147 354T129 306T124 272V270H568Q583 262 583 250T568 230H124V228Q124 207 134 177T167 112T231 48T328 7Q355 1 466 0H570Q583 -10 583 -20Q583 -32 568 -40H471Q464 -40 446 -40T417 -41Q262 -41 172 45Q84 127 84 250Z"></path><path id=MJX-1754-TEX-I-1D443 d="M287 628Q287 635 230 637Q206 637 199 638T192 648Q192 649 194 659Q200 679 203 681T397 683Q587 682 600 680Q664 669 707 631T751 530Q751 453 685 389Q616 321 507 303Q500 302 402 301H307L277 182Q247 66 247 59Q247 55 248 54T255 50T272 48T305 46H336Q342 37 342 35Q342 19 335 5Q330 0 319 0Q316 0 282 1T182 2Q120 2 87 2T51 1Q33 1 33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM645 554Q645 567 643 575T634 597T609 619T560 635Q553 636 480 637Q463 637 445 637T416 636T404 636Q391 635 386 627Q384 621 367 550T332 412T314 344Q314 342 395 342H407H430Q542 342 590 392Q617 419 631 471T645 554Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=msub><g data-mml-node=mi><use data-c=1D45D xlink:href=#MJX-1754-TEX-I-1D45D></use></g><g data-mml-node=mi transform="translate(536,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1754-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(1107.7,0)><use data-c=2208 xlink:href=#MJX-1754-TEX-N-2208></use></g><g data-mml-node=mi transform=translate(2052.5,0)><use data-c=1D443 xlink:href=#MJX-1754-TEX-I-1D443></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><msub><mi>p</mi><mi>i</mi></msub><mo>∈</mo><mi>P</mi></math></mjx-assistive-mml></mjx-container><span>, </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=6.663ex height=2.236ex role=img focusable=false viewBox="0 -694 2944.9 988.2" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.666ex><defs><path id=MJX-1755-TEX-I-1D451 d="M366 683Q367 683 438 688T511 694Q523 694 523 686Q523 679 450 384T375 83T374 68Q374 26 402 26Q411 27 422 35Q443 55 463 131Q469 151 473 152Q475 153 483 153H487H491Q506 153 506 145Q506 140 503 129Q490 79 473 48T445 8T417 -8Q409 -10 393 -10Q359 -10 336 5T306 36L300 51Q299 52 296 50Q294 48 292 46Q233 -10 172 -10Q117 -10 75 30T33 157Q33 205 53 255T101 341Q148 398 195 420T280 442Q336 442 364 400Q369 394 369 396Q370 400 396 505T424 616Q424 629 417 632T378 637H357Q351 643 351 645T353 664Q358 683 366 683ZM352 326Q329 405 277 405Q242 405 210 374T160 293Q131 214 119 129Q119 126 119 118T118 106Q118 61 136 44T179 26Q233 26 290 98L298 109L352 326Z"></path><path id=MJX-1755-TEX-I-1D457 d="M297 596Q297 627 318 644T361 661Q378 661 389 651T403 623Q403 595 384 576T340 557Q322 557 310 567T297 596ZM288 376Q288 405 262 405Q240 405 220 393T185 362T161 325T144 293L137 279Q135 278 121 278H107Q101 284 101 286T105 299Q126 348 164 391T252 441Q253 441 260 441T272 442Q296 441 316 432Q341 418 354 401T367 348V332L318 133Q267 -67 264 -75Q246 -125 194 -164T75 -204Q25 -204 7 -183T-12 -137Q-12 -110 7 -91T53 -71Q70 -71 82 -81T95 -112Q95 -148 63 -167Q69 -168 77 -168Q111 -168 139 -140T182 -74L193 -32Q204 11 219 72T251 197T278 308T289 365Q289 372 288 376Z"></path><path id=MJX-1755-TEX-N-2208 d="M84 250Q84 372 166 450T360 539Q361 539 377 539T419 540T469 540H568Q583 532 583 520Q583 511 570 501L466 500Q355 499 329 494Q280 482 242 458T183 409T147 354T129 306T124 272V270H568Q583 262 583 250T568 230H124V228Q124 207 134 177T167 112T231 48T328 7Q355 1 466 0H570Q583 -10 583 -20Q583 -32 568 -40H471Q464 -40 446 -40T417 -41Q262 -41 172 45Q84 127 84 250Z"></path><path id=MJX-1755-TEX-I-1D437 d="M287 628Q287 635 230 637Q207 637 200 638T193 647Q193 655 197 667T204 682Q206 683 403 683Q570 682 590 682T630 676Q702 659 752 597T803 431Q803 275 696 151T444 3L430 1L236 0H125H72Q48 0 41 2T33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM703 469Q703 507 692 537T666 584T629 613T590 629T555 636Q553 636 541 636T512 636T479 637H436Q392 637 386 627Q384 623 313 339T242 52Q242 48 253 48T330 47Q335 47 349 47T373 46Q499 46 581 128Q617 164 640 212T683 339T703 469Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=msub><g data-mml-node=mi><use data-c=1D451 xlink:href=#MJX-1755-TEX-I-1D451></use></g><g data-mml-node=mi transform="translate(553,-150) scale(0.707)"><use data-c=1D457 xlink:href=#MJX-1755-TEX-I-1D457></use></g></g><g data-mml-node=mo transform=translate(1172.1,0)><use data-c=2208 xlink:href=#MJX-1755-TEX-N-2208></use></g><g data-mml-node=mi transform=translate(2116.9,0)><use data-c=1D437 xlink:href=#MJX-1755-TEX-I-1D437></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><msub><mi>d</mi><mi>j</mi></msub><mo>∈</mo><mi>D</mi></math></mjx-assistive-mml></mjx-container><span>, we define the likelihood of </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=1.878ex height=1.439ex role=img focusable=false viewBox="0 -442 830 636" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.439ex><defs><path id=MJX-1583-TEX-I-1D45D d="M23 287Q24 290 25 295T30 317T40 348T55 381T75 411T101 433T134 442Q209 442 230 378L240 387Q302 442 358 442Q423 442 460 395T497 281Q497 173 421 82T249 -10Q227 -10 210 -4Q199 1 187 11T168 28L161 36Q160 35 139 -51T118 -138Q118 -144 126 -145T163 -148H188Q194 -155 194 -157T191 -175Q188 -187 185 -190T172 -194Q170 -194 161 -194T127 -193T65 -192Q-5 -192 -24 -194H-32Q-39 -187 -39 -183Q-37 -156 -26 -148H-6Q28 -147 33 -136Q36 -130 94 103T155 350Q156 355 156 364Q156 405 131 405Q109 405 94 377T71 316T59 280Q57 278 43 278H29Q23 284 23 287ZM178 102Q200 26 252 26Q282 26 310 49T356 107Q374 141 392 215T411 325V331Q411 405 350 405Q339 405 328 402T306 393T286 380T269 365T254 350T243 336T235 326L232 322Q232 321 229 308T218 264T204 212Q178 106 178 102Z"></path><path id=MJX-1583-TEX-I-1D456 d="M184 600Q184 624 203 642T247 661Q265 661 277 649T290 619Q290 596 270 577T226 557Q211 557 198 567T184 600ZM21 287Q21 295 30 318T54 369T98 420T158 442Q197 442 223 419T250 357Q250 340 236 301T196 196T154 83Q149 61 149 51Q149 26 166 26Q175 26 185 29T208 43T235 78T260 137Q263 149 265 151T282 153Q302 153 302 143Q302 135 293 112T268 61T223 11T161 -11Q129 -11 102 10T74 74Q74 91 79 106T122 220Q160 321 166 341T173 380Q173 404 156 404H154Q124 404 99 371T61 287Q60 286 59 284T58 281T56 279T53 278T49 278T41 278H27Q21 284 21 287Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=msub><g data-mml-node=mi><use data-c=1D45D xlink:href=#MJX-1583-TEX-I-1D45D></use></g><g data-mml-node=mi transform="translate(536,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1583-TEX-I-1D456></use></g></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><msub><mi>p</mi><mi>i</mi></msub></math></mjx-assistive-mml></mjx-container><span> being in Distribution </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=1.873ex height=1.545ex role=img focusable=false viewBox="0 -683 828 683" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:0px><defs><path id=MJX-1586-TEX-I-1D437 d="M287 628Q287 635 230 637Q207 637 200 638T193 647Q193 655 197 667T204 682Q206 683 403 683Q570 682 590 682T630 676Q702 659 752 597T803 431Q803 275 696 151T444 3L430 1L236 0H125H72Q48 0 41 2T33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM703 469Q703 507 692 537T666 584T629 613T590 629T555 636Q553 636 541 636T512 636T479 637H436Q392 637 386 627Q384 623 313 339T242 52Q242 48 253 48T330 47Q335 47 349 47T373 46Q499 46 581 128Q617 164 640 212T683 339T703 469Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D437 xlink:href=#MJX-1586-TEX-I-1D437></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>D</mi></math></mjx-assistive-mml></mjx-container><span> (relative to the other points from </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=1.699ex height=1.545ex role=img focusable=false viewBox="0 -683 751 683" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:0px><defs><path id=MJX-1639-TEX-I-1D443 d="M287 628Q287 635 230 637Q206 637 199 638T192 648Q192 649 194 659Q200 679 203 681T397 683Q587 682 600 680Q664 669 707 631T751 530Q751 453 685 389Q616 321 507 303Q500 302 402 301H307L277 182Q247 66 247 59Q247 55 248 54T255 50T272 48T305 46H336Q342 37 342 35Q342 19 335 5Q330 0 319 0Q316 0 282 1T182 2Q120 2 87 2T51 1Q33 1 33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM645 554Q645 567 643 575T634 597T609 619T560 635Q553 636 480 637Q463 637 445 637T416 636T404 636Q391 635 386 627Q384 621 367 550T332 412T314 344Q314 342 395 342H407H430Q542 342 590 392Q617 419 631 471T645 554Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D443 xlink:href=#MJX-1639-TEX-I-1D443></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>P</mi></math></mjx-assistive-mml></mjx-container><span>) as </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=9.12ex height=2.363ex role=img focusable=false viewBox="0 -750 4030.9 1044.2" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.666ex><defs><path id=MJX-1751-TEX-N-6C d="M42 46H56Q95 46 103 60V68Q103 77 103 91T103 124T104 167T104 217T104 272T104 329Q104 366 104 407T104 482T104 542T103 586T103 603Q100 622 89 628T44 637H26V660Q26 683 28 683L38 684Q48 685 67 686T104 688Q121 689 141 690T171 693T182 694H185V379Q185 62 186 60Q190 52 198 49Q219 46 247 46H263V0H255L232 1Q209 2 183 2T145 3T107 3T57 1L34 0H26V46H42Z"></path><path id=MJX-1751-TEX-N-69 d="M69 609Q69 637 87 653T131 669Q154 667 171 652T188 609Q188 579 171 564T129 549Q104 549 87 564T69 609ZM247 0Q232 3 143 3Q132 3 106 3T56 1L34 0H26V46H42Q70 46 91 49Q100 53 102 60T104 102V205V293Q104 345 102 359T88 378Q74 385 41 385H30V408Q30 431 32 431L42 432Q52 433 70 434T106 436Q123 437 142 438T171 441T182 442H185V62Q190 52 197 50T232 46H255V0H247Z"></path><path id=MJX-1751-TEX-N-6B d="M36 46H50Q89 46 97 60V68Q97 77 97 91T97 124T98 167T98 217T98 272T98 329Q98 366 98 407T98 482T98 542T97 586T97 603Q94 622 83 628T38 637H20V660Q20 683 22 683L32 684Q42 685 61 686T98 688Q115 689 135 690T165 693T176 694H179V463L180 233L240 287Q300 341 304 347Q310 356 310 364Q310 383 289 385H284V431H293Q308 428 412 428Q475 428 484 431H489V385H476Q407 380 360 341Q286 278 286 274Q286 273 349 181T420 79Q434 60 451 53T500 46H511V0H505Q496 3 418 3Q322 3 307 0H299V46H306Q330 48 330 65Q330 72 326 79Q323 84 276 153T228 222L176 176V120V84Q176 65 178 59T189 49Q210 46 238 46H254V0H246Q231 3 137 3T28 0H20V46H36Z"></path><path id=MJX-1751-TEX-N-28 d="M94 250Q94 319 104 381T127 488T164 576T202 643T244 695T277 729T302 750H315H319Q333 750 333 741Q333 738 316 720T275 667T226 581T184 443T167 250T184 58T225 -81T274 -167T316 -220T333 -241Q333 -250 318 -250H315H302L274 -226Q180 -141 137 -14T94 250Z"></path><path id=MJX-1751-TEX-I-1D45D d="M23 287Q24 290 25 295T30 317T40 348T55 381T75 411T101 433T134 442Q209 442 230 378L240 387Q302 442 358 442Q423 442 460 395T497 281Q497 173 421 82T249 -10Q227 -10 210 -4Q199 1 187 11T168 28L161 36Q160 35 139 -51T118 -138Q118 -144 126 -145T163 -148H188Q194 -155 194 -157T191 -175Q188 -187 185 -190T172 -194Q170 -194 161 -194T127 -193T65 -192Q-5 -192 -24 -194H-32Q-39 -187 -39 -183Q-37 -156 -26 -148H-6Q28 -147 33 -136Q36 -130 94 103T155 350Q156 355 156 364Q156 405 131 405Q109 405 94 377T71 316T59 280Q57 278 43 278H29Q23 284 23 287ZM178 102Q200 26 252 26Q282 26 310 49T356 107Q374 141 392 215T411 325V331Q411 405 350 405Q339 405 328 402T306 393T286 380T269 365T254 350T243 336T235 326L232 322Q232 321 229 308T218 264T204 212Q178 106 178 102Z"></path><path id=MJX-1751-TEX-I-1D456 d="M184 600Q184 624 203 642T247 661Q265 661 277 649T290 619Q290 596 270 577T226 557Q211 557 198 567T184 600ZM21 287Q21 295 30 318T54 369T98 420T158 442Q197 442 223 419T250 357Q250 340 236 301T196 196T154 83Q149 61 149 51Q149 26 166 26Q175 26 185 29T208 43T235 78T260 137Q263 149 265 151T282 153Q302 153 302 143Q302 135 293 112T268 61T223 11T161 -11Q129 -11 102 10T74 74Q74 91 79 106T122 220Q160 321 166 341T173 380Q173 404 156 404H154Q124 404 99 371T61 287Q60 286 59 284T58 281T56 279T53 278T49 278T41 278H27Q21 284 21 287Z"></path><path id=MJX-1751-TEX-N-2C d="M78 35T78 60T94 103T137 121Q165 121 187 96T210 8Q210 -27 201 -60T180 -117T154 -158T130 -185T117 -194Q113 -194 104 -185T95 -172Q95 -168 106 -156T131 -126T157 -76T173 -3V9L172 8Q170 7 167 6T161 3T152 1T140 0Q113 0 96 17Z"></path><path id=MJX-1751-TEX-I-1D451 d="M366 683Q367 683 438 688T511 694Q523 694 523 686Q523 679 450 384T375 83T374 68Q374 26 402 26Q411 27 422 35Q443 55 463 131Q469 151 473 152Q475 153 483 153H487H491Q506 153 506 145Q506 140 503 129Q490 79 473 48T445 8T417 -8Q409 -10 393 -10Q359 -10 336 5T306 36L300 51Q299 52 296 50Q294 48 292 46Q233 -10 172 -10Q117 -10 75 30T33 157Q33 205 53 255T101 341Q148 398 195 420T280 442Q336 442 364 400Q369 394 369 396Q370 400 396 505T424 616Q424 629 417 632T378 637H357Q351 643 351 645T353 664Q358 683 366 683ZM352 326Q329 405 277 405Q242 405 210 374T160 293Q131 214 119 129Q119 126 119 118T118 106Q118 61 136 44T179 26Q233 26 290 98L298 109L352 326Z"></path><path id=MJX-1751-TEX-I-1D457 d="M297 596Q297 627 318 644T361 661Q378 661 389 651T403 623Q403 595 384 576T340 557Q322 557 310 567T297 596ZM288 376Q288 405 262 405Q240 405 220 393T185 362T161 325T144 293L137 279Q135 278 121 278H107Q101 284 101 286T105 299Q126 348 164 391T252 441Q253 441 260 441T272 442Q296 441 316 432Q341 418 354 401T367 348V332L318 133Q267 -67 264 -75Q246 -125 194 -164T75 -204Q25 -204 7 -183T-12 -137Q-12 -110 7 -91T53 -71Q70 -71 82 -81T95 -112Q95 -148 63 -167Q69 -168 77 -168Q111 -168 139 -140T182 -74L193 -32Q204 11 219 72T251 197T278 308T289 365Q289 372 288 376Z"></path><path id=MJX-1751-TEX-N-29 d="M60 749L64 750Q69 750 74 750H86L114 726Q208 641 251 514T294 250Q294 182 284 119T261 12T224 -76T186 -143T145 -194T113 -227T90 -246Q87 -249 86 -250H74Q66 -250 63 -250T58 -247T55 -238Q56 -237 66 -225Q221 -64 221 250T66 725Q56 737 55 738Q55 746 60 749Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mtext><use data-c=6C xlink:href=#MJX-1751-TEX-N-6C></use><use data-c=69 xlink:href=#MJX-1751-TEX-N-69 transform=translate(278,0)></use><use data-c=6B xlink:href=#MJX-1751-TEX-N-6B transform=translate(556,0)></use></g><g data-mml-node=mo transform=translate(1084,0)><use data-c=28 xlink:href=#MJX-1751-TEX-N-28></use></g><g data-mml-node=msub transform=translate(1473,0)><g data-mml-node=mi><use data-c=1D45D xlink:href=#MJX-1751-TEX-I-1D45D></use></g><g data-mml-node=mi transform="translate(536,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1751-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(2303,0)><use data-c=2C xlink:href=#MJX-1751-TEX-N-2C></use></g><g data-mml-node=msub transform=translate(2747.6,0)><g data-mml-node=mi><use data-c=1D451 xlink:href=#MJX-1751-TEX-I-1D451></use></g><g data-mml-node=mi transform="translate(553,-150) scale(0.707)"><use data-c=1D457 xlink:href=#MJX-1751-TEX-I-1D457></use></g></g><g data-mml-node=mo transform=translate(3641.9,0)><use data-c=29 xlink:href=#MJX-1751-TEX-N-29></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mtext>lik</mtext><mo stretchy=false>(</mo><msub><mi>p</mi><mi>i</mi></msub><mo>,</mo><msub><mi>d</mi><mi>j</mi></msub><mo stretchy=false>)</mo></math></mjx-assistive-mml></mjx-container><span>.</span><p><span>We aim to find a subset </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=6.415ex height=2.554ex role=img focusable=false viewBox="0 -991 2835.6 1129" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.312ex><defs><path id=MJX-1767-TEX-I-1D443 d="M287 628Q287 635 230 637Q206 637 199 638T192 648Q192 649 194 659Q200 679 203 681T397 683Q587 682 600 680Q664 669 707 631T751 530Q751 453 685 389Q616 321 507 303Q500 302 402 301H307L277 182Q247 66 247 59Q247 55 248 54T255 50T272 48T305 46H336Q342 37 342 35Q342 19 335 5Q330 0 319 0Q316 0 282 1T182 2Q120 2 87 2T51 1Q33 1 33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM645 554Q645 567 643 575T634 597T609 619T560 635Q553 636 480 637Q463 637 445 637T416 636T404 636Q391 635 386 627Q384 621 367 550T332 412T314 344Q314 342 395 342H407H430Q542 342 590 392Q617 419 631 471T645 554Z"></path><path id=MJX-1767-TEX-N-7E d="M179 251Q164 251 151 245T131 234T111 215L97 227L83 238Q83 239 95 253T121 283T142 304Q165 318 187 318T253 300T320 282Q335 282 348 288T368 299T388 318L402 306L416 295Q375 236 344 222Q330 215 313 215Q292 215 248 233T179 251Z"></path><path id=MJX-1767-TEX-N-2286 d="M84 346Q84 468 166 546T360 635Q361 635 370 635T395 635T430 636T475 636T524 636H679Q694 628 694 616Q694 607 681 597L522 596H470H441Q366 596 338 592T266 568Q244 557 224 542T179 500T139 433T124 346V341Q124 253 185 185Q244 121 328 103Q348 98 366 98T522 96H681Q694 86 694 76Q694 64 679 56H526Q510 56 480 56T434 55Q350 55 289 71T172 141Q84 223 84 346ZM104 -131T104 -118T118 -98H679Q694 -106 694 -118T679 -138H118Q104 -131 104 -118Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=TeXAtom data-mjx-texclass=ORD><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D443 xlink:href=#MJX-1767-TEX-I-1D443></use></g><g data-mml-node=mo transform="translate(486,573) translate(-250 0)"><use data-c=7E xlink:href=#MJX-1767-TEX-N-7E></use></g></g></g><g data-mml-node=mo transform=translate(1028.8,0)><use data-c=2286 xlink:href=#MJX-1767-TEX-N-2286></use></g><g data-mml-node=mi transform=translate(2084.6,0)><use data-c=1D443 xlink:href=#MJX-1767-TEX-I-1D443></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mrow data-mjx-texclass=ORD><mover><mi>P</mi><mo stretchy=false>~</mo></mover></mrow><mo>⊆</mo><mi>P</mi></math></mjx-assistive-mml></mjx-container><span> with </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=1.061ex height=1.023ex role=img focusable=false viewBox="0 -442 469 452" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.023ex><defs><path id=MJX-1511-TEX-I-1D460 d="M131 289Q131 321 147 354T203 415T300 442Q362 442 390 415T419 355Q419 323 402 308T364 292Q351 292 340 300T328 326Q328 342 337 354T354 372T367 378Q368 378 368 379Q368 382 361 388T336 399T297 405Q249 405 227 379T204 326Q204 301 223 291T278 274T330 259Q396 230 396 163Q396 135 385 107T352 51T289 7T195 -10Q118 -10 86 19T53 87Q53 126 74 143T118 160Q133 160 146 151T160 120Q160 94 142 76T111 58Q109 57 108 57T107 55Q108 52 115 47T146 34T201 27Q237 27 263 38T301 66T318 97T323 122Q323 150 302 164T254 181T195 196T148 231Q131 256 131 289Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D460 xlink:href=#MJX-1511-TEX-I-1D460></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>s</mi></math></mjx-assistive-mml></mjx-container><span> elements, such that we obtain a mapping </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=9.787ex height=2.262ex role=img focusable=false viewBox="0 -750 4326.1 1000" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.566ex><defs><path id=MJX-1768-TEX-N-28 d="M94 250Q94 319 104 381T127 488T164 576T202 643T244 695T277 729T302 750H315H319Q333 750 333 741Q333 738 316 720T275 667T226 581T184 443T167 250T184 58T225 -81T274 -167T316 -220T333 -241Q333 -250 318 -250H315H302L274 -226Q180 -141 137 -14T94 250Z"></path><path id=MJX-1768-TEX-I-1D45D d="M23 287Q24 290 25 295T30 317T40 348T55 381T75 411T101 433T134 442Q209 442 230 378L240 387Q302 442 358 442Q423 442 460 395T497 281Q497 173 421 82T249 -10Q227 -10 210 -4Q199 1 187 11T168 28L161 36Q160 35 139 -51T118 -138Q118 -144 126 -145T163 -148H188Q194 -155 194 -157T191 -175Q188 -187 185 -190T172 -194Q170 -194 161 -194T127 -193T65 -192Q-5 -192 -24 -194H-32Q-39 -187 -39 -183Q-37 -156 -26 -148H-6Q28 -147 33 -136Q36 -130 94 103T155 350Q156 355 156 364Q156 405 131 405Q109 405 94 377T71 316T59 280Q57 278 43 278H29Q23 284 23 287ZM178 102Q200 26 252 26Q282 26 310 49T356 107Q374 141 392 215T411 325V331Q411 405 350 405Q339 405 328 402T306 393T286 380T269 365T254 350T243 336T235 326L232 322Q232 321 229 308T218 264T204 212Q178 106 178 102Z"></path><path id=MJX-1768-TEX-N-7E d="M179 251Q164 251 151 245T131 234T111 215L97 227L83 238Q83 239 95 253T121 283T142 304Q165 318 187 318T253 300T320 282Q335 282 348 288T368 299T388 318L402 306L416 295Q375 236 344 222Q330 215 313 215Q292 215 248 233T179 251Z"></path><path id=MJX-1768-TEX-I-1D456 d="M184 600Q184 624 203 642T247 661Q265 661 277 649T290 619Q290 596 270 577T226 557Q211 557 198 567T184 600ZM21 287Q21 295 30 318T54 369T98 420T158 442Q197 442 223 419T250 357Q250 340 236 301T196 196T154 83Q149 61 149 51Q149 26 166 26Q175 26 185 29T208 43T235 78T260 137Q263 149 265 151T282 153Q302 153 302 143Q302 135 293 112T268 61T223 11T161 -11Q129 -11 102 10T74 74Q74 91 79 106T122 220Q160 321 166 341T173 380Q173 404 156 404H154Q124 404 99 371T61 287Q60 286 59 284T58 281T56 279T53 278T49 278T41 278H27Q21 284 21 287Z"></path><path id=MJX-1768-TEX-N-2C d="M78 35T78 60T94 103T137 121Q165 121 187 96T210 8Q210 -27 201 -60T180 -117T154 -158T130 -185T117 -194Q113 -194 104 -185T95 -172Q95 -168 106 -156T131 -126T157 -76T173 -3V9L172 8Q170 7 167 6T161 3T152 1T140 0Q113 0 96 17Z"></path><path id=MJX-1768-TEX-I-1D451 d="M366 683Q367 683 438 688T511 694Q523 694 523 686Q523 679 450 384T375 83T374 68Q374 26 402 26Q411 27 422 35Q443 55 463 131Q469 151 473 152Q475 153 483 153H487H491Q506 153 506 145Q506 140 503 129Q490 79 473 48T445 8T417 -8Q409 -10 393 -10Q359 -10 336 5T306 36L300 51Q299 52 296 50Q294 48 292 46Q233 -10 172 -10Q117 -10 75 30T33 157Q33 205 53 255T101 341Q148 398 195 420T280 442Q336 442 364 400Q369 394 369 396Q370 400 396 505T424 616Q424 629 417 632T378 637H357Q351 643 351 645T353 664Q358 683 366 683ZM352 326Q329 405 277 405Q242 405 210 374T160 293Q131 214 119 129Q119 126 119 118T118 106Q118 61 136 44T179 26Q233 26 290 98L298 109L352 326Z"></path><path id=MJX-1768-TEX-N-29 d="M60 749L64 750Q69 750 74 750H86L114 726Q208 641 251 514T294 250Q294 182 284 119T261 12T224 -76T186 -143T145 -194T113 -227T90 -246Q87 -249 86 -250H74Q66 -250 63 -250T58 -247T55 -238Q56 -237 66 -225Q221 -64 221 250T66 725Q56 737 55 738Q55 746 60 749Z"></path><path id=MJX-1768-TEX-N-2208 d="M84 250Q84 372 166 450T360 539Q361 539 377 539T419 540T469 540H568Q583 532 583 520Q583 511 570 501L466 500Q355 499 329 494Q280 482 242 458T183 409T147 354T129 306T124 272V270H568Q583 262 583 250T568 230H124V228Q124 207 134 177T167 112T231 48T328 7Q355 1 466 0H570Q583 -10 583 -20Q583 -32 568 -40H471Q464 -40 446 -40T417 -41Q262 -41 172 45Q84 127 84 250Z"></path><path id=MJX-1768-TEX-I-1D441 d="M234 637Q231 637 226 637Q201 637 196 638T191 649Q191 676 202 682Q204 683 299 683Q376 683 387 683T401 677Q612 181 616 168L670 381Q723 592 723 606Q723 633 659 637Q635 637 635 648Q635 650 637 660Q641 676 643 679T653 683Q656 683 684 682T767 680Q817 680 843 681T873 682Q888 682 888 672Q888 650 880 642Q878 637 858 637Q787 633 769 597L620 7Q618 0 599 0Q585 0 582 2Q579 5 453 305L326 604L261 344Q196 88 196 79Q201 46 268 46H278Q284 41 284 38T282 19Q278 6 272 0H259Q228 2 151 2Q123 2 100 2T63 2T46 1Q31 1 31 10Q31 14 34 26T39 40Q41 46 62 46Q130 49 150 85Q154 91 221 362L289 634Q287 635 234 637Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mo><use data-c=28 xlink:href=#MJX-1768-TEX-N-28></use></g><g data-mml-node=msub transform=translate(389,0)><g data-mml-node=TeXAtom data-mjx-texclass=ORD><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D45D xlink:href=#MJX-1768-TEX-I-1D45D></use></g><g data-mml-node=mo transform="translate(334.8,332) translate(-250 0)"><use data-c=7E xlink:href=#MJX-1768-TEX-N-7E></use></g></g></g><g data-mml-node=mi transform="translate(536,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1768-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(1219,0)><use data-c=2C xlink:href=#MJX-1768-TEX-N-2C></use></g><g data-mml-node=msub transform=translate(1663.6,0)><g data-mml-node=mi><use data-c=1D451 xlink:href=#MJX-1768-TEX-I-1D451></use></g><g data-mml-node=mi transform="translate(553,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1768-TEX-I-1D456></use></g></g><g data-mml-node=msub transform=translate(2510.6,0)><g data-mml-node=mo><use data-c=29 xlink:href=#MJX-1768-TEX-N-29></use></g><g data-mml-node=TeXAtom transform="translate(422,-150) scale(0.707)" data-mjx-texclass=ORD><g data-mml-node=mi><use data-c=1D456 xlink:href=#MJX-1768-TEX-I-1D456></use></g><g data-mml-node=mo transform=translate(345,0)><use data-c=2208 xlink:href=#MJX-1768-TEX-N-2208></use></g><g data-mml-node=mi transform=translate(1012,0)><use data-c=1D441 xlink:href=#MJX-1768-TEX-I-1D441></use></g></g></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mo stretchy=false>(</mo><msub><mrow data-mjx-texclass=ORD><mover><mi>p</mi><mo stretchy=false>~</mo></mover></mrow><mi>i</mi></msub><mo>,</mo><msub><mi>d</mi><mi>i</mi></msub><msub><mo stretchy=false>)</mo><mrow data-mjx-texclass=ORD><mi>i</mi><mo>∈</mo><mi>N</mi></mrow></msub></math></mjx-assistive-mml></mjx-container><span> where</span><div contenteditable=false spellcheck=false class="mathjax-block md-end-block md-math-block md-rawblock" id=mathjax-n9 cid=n9 mdtype=math_block data-math-tag-before=0 data-math-tag-after=1 data-math-labels=[]><div class=md-math-container><mjx-container class=MathJax jax=SVG display=true width=full style=min-width:55.965ex;position:relative><svg xmlns=http://www.w3.org/2000/svg width=100% height=6.74ex role=img focusable=false xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-2.804ex;min-width:55.965ex><defs><path id=MJX-1773-TEX-LO-220F d="M220 812Q220 813 218 819T214 829T208 840T199 853T185 866T166 878T140 887T107 893T66 896H56V950H1221V896H1211Q1080 896 1058 812V-311Q1076 -396 1211 -396H1221V-450H725V-396H735Q864 -396 888 -314Q889 -312 889 -311V896H388V292L389 -311Q405 -396 542 -396H552V-450H56V-396H66Q195 -396 219 -314Q220 -312 220 -311V812Z"></path><path id=MJX-1773-TEX-I-1D456 d="M184 600Q184 624 203 642T247 661Q265 661 277 649T290 619Q290 596 270 577T226 557Q211 557 198 567T184 600ZM21 287Q21 295 30 318T54 369T98 420T158 442Q197 442 223 419T250 357Q250 340 236 301T196 196T154 83Q149 61 149 51Q149 26 166 26Q175 26 185 29T208 43T235 78T260 137Q263 149 265 151T282 153Q302 153 302 143Q302 135 293 112T268 61T223 11T161 -11Q129 -11 102 10T74 74Q74 91 79 106T122 220Q160 321 166 341T173 380Q173 404 156 404H154Q124 404 99 371T61 287Q60 286 59 284T58 281T56 279T53 278T49 278T41 278H27Q21 284 21 287Z"></path><path id=MJX-1773-TEX-N-3D d="M56 347Q56 360 70 367H707Q722 359 722 347Q722 336 708 328L390 327H72Q56 332 56 347ZM56 153Q56 168 72 173H708Q722 163 722 153Q722 140 707 133H70Q56 140 56 153Z"></path><path id=MJX-1773-TEX-N-31 d="M213 578L200 573Q186 568 160 563T102 556H83V602H102Q149 604 189 617T245 641T273 663Q275 666 285 666Q294 666 302 660V361L303 61Q310 54 315 52T339 48T401 46H427V0H416Q395 3 257 3Q121 3 100 0H88V46H114Q136 46 152 46T177 47T193 50T201 52T207 57T213 61V578Z"></path><path id=MJX-1773-TEX-I-1D441 d="M234 637Q231 637 226 637Q201 637 196 638T191 649Q191 676 202 682Q204 683 299 683Q376 683 387 683T401 677Q612 181 616 168L670 381Q723 592 723 606Q723 633 659 637Q635 637 635 648Q635 650 637 660Q641 676 643 679T653 683Q656 683 684 682T767 680Q817 680 843 681T873 682Q888 682 888 672Q888 650 880 642Q878 637 858 637Q787 633 769 597L620 7Q618 0 599 0Q585 0 582 2Q579 5 453 305L326 604L261 344Q196 88 196 79Q201 46 268 46H278Q284 41 284 38T282 19Q278 6 272 0H259Q228 2 151 2Q123 2 100 2T63 2T46 1Q31 1 31 10Q31 14 34 26T39 40Q41 46 62 46Q130 49 150 85Q154 91 221 362L289 634Q287 635 234 637Z"></path><path id=MJX-1773-TEX-N-6C d="M42 46H56Q95 46 103 60V68Q103 77 103 91T103 124T104 167T104 217T104 272T104 329Q104 366 104 407T104 482T104 542T103 586T103 603Q100 622 89 628T44 637H26V660Q26 683 28 683L38 684Q48 685 67 686T104 688Q121 689 141 690T171 693T182 694H185V379Q185 62 186 60Q190 52 198 49Q219 46 247 46H263V0H255L232 1Q209 2 183 2T145 3T107 3T57 1L34 0H26V46H42Z"></path><path id=MJX-1773-TEX-N-69 d="M69 609Q69 637 87 653T131 669Q154 667 171 652T188 609Q188 579 171 564T129 549Q104 549 87 564T69 609ZM247 0Q232 3 143 3Q132 3 106 3T56 1L34 0H26V46H42Q70 46 91 49Q100 53 102 60T104 102V205V293Q104 345 102 359T88 378Q74 385 41 385H30V408Q30 431 32 431L42 432Q52 433 70 434T106 436Q123 437 142 438T171 441T182 442H185V62Q190 52 197 50T232 46H255V0H247Z"></path><path id=MJX-1773-TEX-N-6B d="M36 46H50Q89 46 97 60V68Q97 77 97 91T97 124T98 167T98 217T98 272T98 329Q98 366 98 407T98 482T98 542T97 586T97 603Q94 622 83 628T38 637H20V660Q20 683 22 683L32 684Q42 685 61 686T98 688Q115 689 135 690T165 693T176 694H179V463L180 233L240 287Q300 341 304 347Q310 356 310 364Q310 383 289 385H284V431H293Q308 428 412 428Q475 428 484 431H489V385H476Q407 380 360 341Q286 278 286 274Q286 273 349 181T420 79Q434 60 451 53T500 46H511V0H505Q496 3 418 3Q322 3 307 0H299V46H306Q330 48 330 65Q330 72 326 79Q323 84 276 153T228 222L176 176V120V84Q176 65 178 59T189 49Q210 46 238 46H254V0H246Q231 3 137 3T28 0H20V46H36Z"></path><path id=MJX-1773-TEX-N-28 d="M94 250Q94 319 104 381T127 488T164 576T202 643T244 695T277 729T302 750H315H319Q333 750 333 741Q333 738 316 720T275 667T226 581T184 443T167 250T184 58T225 -81T274 -167T316 -220T333 -241Q333 -250 318 -250H315H302L274 -226Q180 -141 137 -14T94 250Z"></path><path id=MJX-1773-TEX-I-1D45D d="M23 287Q24 290 25 295T30 317T40 348T55 381T75 411T101 433T134 442Q209 442 230 378L240 387Q302 442 358 442Q423 442 460 395T497 281Q497 173 421 82T249 -10Q227 -10 210 -4Q199 1 187 11T168 28L161 36Q160 35 139 -51T118 -138Q118 -144 126 -145T163 -148H188Q194 -155 194 -157T191 -175Q188 -187 185 -190T172 -194Q170 -194 161 -194T127 -193T65 -192Q-5 -192 -24 -194H-32Q-39 -187 -39 -183Q-37 -156 -26 -148H-6Q28 -147 33 -136Q36 -130 94 103T155 350Q156 355 156 364Q156 405 131 405Q109 405 94 377T71 316T59 280Q57 278 43 278H29Q23 284 23 287ZM178 102Q200 26 252 26Q282 26 310 49T356 107Q374 141 392 215T411 325V331Q411 405 350 405Q339 405 328 402T306 393T286 380T269 365T254 350T243 336T235 326L232 322Q232 321 229 308T218 264T204 212Q178 106 178 102Z"></path><path id=MJX-1773-TEX-N-7E d="M179 251Q164 251 151 245T131 234T111 215L97 227L83 238Q83 239 95 253T121 283T142 304Q165 318 187 318T253 300T320 282Q335 282 348 288T368 299T388 318L402 306L416 295Q375 236 344 222Q330 215 313 215Q292 215 248 233T179 251Z"></path><path id=MJX-1773-TEX-N-2C d="M78 35T78 60T94 103T137 121Q165 121 187 96T210 8Q210 -27 201 -60T180 -117T154 -158T130 -185T117 -194Q113 -194 104 -185T95 -172Q95 -168 106 -156T131 -126T157 -76T173 -3V9L172 8Q170 7 167 6T161 3T152 1T140 0Q113 0 96 17Z"></path><path id=MJX-1773-TEX-I-1D451 d="M366 683Q367 683 438 688T511 694Q523 694 523 686Q523 679 450 384T375 83T374 68Q374 26 402 26Q411 27 422 35Q443 55 463 131Q469 151 473 152Q475 153 483 153H487H491Q506 153 506 145Q506 140 503 129Q490 79 473 48T445 8T417 -8Q409 -10 393 -10Q359 -10 336 5T306 36L300 51Q299 52 296 50Q294 48 292 46Q233 -10 172 -10Q117 -10 75 30T33 157Q33 205 53 255T101 341Q148 398 195 420T280 442Q336 442 364 400Q369 394 369 396Q370 400 396 505T424 616Q424 629 417 632T378 637H357Q351 643 351 645T353 664Q358 683 366 683ZM352 326Q329 405 277 405Q242 405 210 374T160 293Q131 214 119 129Q119 126 119 118T118 106Q118 61 136 44T179 26Q233 26 290 98L298 109L352 326Z"></path><path id=MJX-1773-TEX-N-29 d="M60 749L64 750Q69 750 74 750H86L114 726Q208 641 251 514T294 250Q294 182 284 119T261 12T224 -76T186 -143T145 -194T113 -227T90 -246Q87 -249 86 -250H74Q66 -250 63 -250T58 -247T55 -238Q56 -237 66 -225Q221 -64 221 250T66 725Q56 737 55 738Q55 746 60 749Z"></path><path id=MJX-1773-TEX-N-2265 d="M83 616Q83 624 89 630T99 636Q107 636 253 568T543 431T687 361Q694 356 694 346T687 331Q685 329 395 192L107 56H101Q83 58 83 76Q83 77 83 79Q82 86 98 95Q117 105 248 167Q326 204 378 228L626 346L360 472Q291 505 200 548Q112 589 98 597T83 616ZM84 -118Q84 -108 99 -98H678Q694 -104 694 -118Q694 -130 679 -138H98Q84 -131 84 -118Z"></path><path id=MJX-1773-TEX-N-5E d="M112 560L249 694L257 686Q387 562 387 560L361 531Q359 532 303 581L250 627L195 580Q182 569 169 557T148 538L140 532Q138 530 125 546L112 560Z"></path><path id=MJX-1773-TEX-N-2200 d="M0 673Q0 684 7 689T20 694Q32 694 38 680T82 567L126 451H430L473 566Q483 593 494 622T512 668T519 685Q524 694 538 694Q556 692 556 674Q556 670 426 329T293 -15Q288 -22 278 -22T263 -15Q260 -11 131 328T0 673ZM414 410Q414 411 278 411T142 410L278 55L414 410Z"></path><path id=MJX-1773-TEX-I-1D443 d="M287 628Q287 635 230 637Q206 637 199 638T192 648Q192 649 194 659Q200 679 203 681T397 683Q587 682 600 680Q664 669 707 631T751 530Q751 453 685 389Q616 321 507 303Q500 302 402 301H307L277 182Q247 66 247 59Q247 55 248 54T255 50T272 48T305 46H336Q342 37 342 35Q342 19 335 5Q330 0 319 0Q316 0 282 1T182 2Q120 2 87 2T51 1Q33 1 33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM645 554Q645 567 643 575T634 597T609 619T560 635Q553 636 480 637Q463 637 445 637T416 636T404 636Q391 635 386 627Q384 621 367 550T332 412T314 344Q314 342 395 342H407H430Q542 342 590 392Q617 419 631 471T645 554Z"></path><path id=MJX-1773-TEX-N-2286 d="M84 346Q84 468 166 546T360 635Q361 635 370 635T395 635T430 636T475 636T524 636H679Q694 628 694 616Q694 607 681 597L522 596H470H441Q366 596 338 592T266 568Q244 557 224 542T179 500T139 433T124 346V341Q124 253 185 185Q244 121 328 103Q348 98 366 98T522 96H681Q694 86 694 76Q694 64 679 56H526Q510 56 480 56T434 55Q350 55 289 71T172 141Q84 223 84 346ZM104 -131T104 -118T118 -98H679Q694 -106 694 -118T679 -138H118Q104 -131 104 -118Z"></path><path id=MJX-1773-TEX-N-7C d="M139 -249H137Q125 -249 119 -235V251L120 737Q130 750 139 750Q152 750 159 735V-235Q151 -249 141 -249H139Z"></path><path id=MJX-1773-TEX-I-1D460 d="M131 289Q131 321 147 354T203 415T300 442Q362 442 390 415T419 355Q419 323 402 308T364 292Q351 292 340 300T328 326Q328 342 337 354T354 372T367 378Q368 378 368 379Q368 382 361 388T336 399T297 405Q249 405 227 379T204 326Q204 301 223 291T278 274T330 259Q396 230 396 163Q396 135 385 107T352 51T289 7T195 -10Q118 -10 86 19T53 87Q53 126 74 143T118 160Q133 160 146 151T160 120Q160 94 142 76T111 58Q109 57 108 57T107 55Q108 52 115 47T146 34T201 27Q237 27 263 38T301 66T318 97T323 122Q323 150 302 164T254 181T195 196T148 231Q131 256 131 289Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform="scale(0.019042,-0.019042) translate(0, -1739.4)"><g data-mml-node=math><g data-mml-node=mtable transform="translate(2078,0) translate(-2078,0)"><g transform="translate(0 1739.4) matrix(1 0 0 -1 0 0) scale(52.51)"><svg data-table=true preserveAspectRatio=xMidYMid viewBox="10290.2 -1739.4 1 2978.9"><g transform="matrix(1 0 0 -1 0 0)"><g data-mml-node=mlabeledtr transform=translate(0,6.5)><g data-mml-node=mtd><g data-mml-node=munderover><g data-mml-node=mo><use data-c=220F xlink:href=#MJX-1773-TEX-LO-220F></use></g><g data-mml-node=TeXAtom transform="translate(65.2,-1087.9) scale(0.707)" data-mjx-texclass=ORD><g data-mml-node=mi><use data-c=1D456 xlink:href=#MJX-1773-TEX-I-1D456></use></g><g data-mml-node=mo transform=translate(345,0)><use data-c=3D xlink:href=#MJX-1773-TEX-N-3D></use></g><g data-mml-node=mn transform=translate(1123,0)><use data-c=31 xlink:href=#MJX-1773-TEX-N-31></use></g></g><g data-mml-node=mi transform="translate(325,1150) scale(0.707)"><use data-c=1D441 xlink:href=#MJX-1773-TEX-I-1D441></use></g></g><g data-mml-node=mtext transform=translate(1444.7,0)><use data-c=6C xlink:href=#MJX-1773-TEX-N-6C></use><use data-c=69 xlink:href=#MJX-1773-TEX-N-69 transform=translate(278,0)></use><use data-c=6B xlink:href=#MJX-1773-TEX-N-6B transform=translate(556,0)></use></g><g data-mml-node=mo transform=translate(2528.7,0)><use data-c=28 xlink:href=#MJX-1773-TEX-N-28></use></g><g data-mml-node=msub transform=translate(2917.7,0)><g data-mml-node=TeXAtom data-mjx-texclass=ORD><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D45D xlink:href=#MJX-1773-TEX-I-1D45D></use></g><g data-mml-node=mo transform="translate(334.8,332) translate(-250 0)"><use data-c=7E xlink:href=#MJX-1773-TEX-N-7E></use></g></g></g><g data-mml-node=mi transform="translate(536,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1773-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(3747.6,0)><use data-c=2C xlink:href=#MJX-1773-TEX-N-2C></use></g><g data-mml-node=msub transform=translate(4192.3,0)><g data-mml-node=mi><use data-c=1D451 xlink:href=#MJX-1773-TEX-I-1D451></use></g><g data-mml-node=mi transform="translate(553,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1773-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(5039.2,0)><use data-c=29 xlink:href=#MJX-1773-TEX-N-29></use></g><g data-mml-node=mo transform=translate(5706,0)><use data-c=2265 xlink:href=#MJX-1773-TEX-N-2265></use></g><g data-mml-node=munderover transform=translate(6761.8,0)><g data-mml-node=mo><use data-c=220F xlink:href=#MJX-1773-TEX-LO-220F></use></g><g data-mml-node=TeXAtom transform="translate(65.2,-1087.9) scale(0.707)" data-mjx-texclass=ORD><g data-mml-node=mi><use data-c=1D456 xlink:href=#MJX-1773-TEX-I-1D456></use></g><g data-mml-node=mo transform=translate(345,0)><use data-c=3D xlink:href=#MJX-1773-TEX-N-3D></use></g><g data-mml-node=mn transform=translate(1123,0)><use data-c=31 xlink:href=#MJX-1773-TEX-N-31></use></g></g><g data-mml-node=mi transform="translate(325,1150) scale(0.707)"><use data-c=1D441 xlink:href=#MJX-1773-TEX-I-1D441></use></g></g><g data-mml-node=mtext transform=translate(8206.5,0)><use data-c=6C xlink:href=#MJX-1773-TEX-N-6C></use><use data-c=69 xlink:href=#MJX-1773-TEX-N-69 transform=translate(278,0)></use><use data-c=6B xlink:href=#MJX-1773-TEX-N-6B transform=translate(556,0)></use></g><g data-mml-node=mo transform=translate(9290.5,0)><use data-c=28 xlink:href=#MJX-1773-TEX-N-28></use></g><g data-mml-node=msub transform=translate(9679.5,0)><g data-mml-node=TeXAtom data-mjx-texclass=ORD><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D45D xlink:href=#MJX-1773-TEX-I-1D45D></use></g><g data-mml-node=mo transform="translate(334.8,16) translate(-250 0)"><use data-c=5E xlink:href=#MJX-1773-TEX-N-5E></use></g></g></g><g data-mml-node=mi transform="translate(536,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1773-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(10509.4,0)><use data-c=2C xlink:href=#MJX-1773-TEX-N-2C></use></g><g data-mml-node=msub transform=translate(10954.1,0)><g data-mml-node=mi><use data-c=1D451 xlink:href=#MJX-1773-TEX-I-1D451></use></g><g data-mml-node=mi transform="translate(553,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1773-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(11801,0)><use data-c=29 xlink:href=#MJX-1773-TEX-N-29></use></g><g data-mml-node=mo transform=translate(12190,0)><use data-c=2C xlink:href=#MJX-1773-TEX-N-2C></use></g><g data-mml-node=mstyle transform=translate(12468,0)><g data-mml-node=mspace></g></g><g data-mml-node=mi transform=translate(13634.7,0)><use data-c=2200 xlink:href=#MJX-1773-TEX-N-2200></use></g><g data-mml-node=TeXAtom data-mjx-texclass=ORD transform=translate(14190.7,0)><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D443 xlink:href=#MJX-1773-TEX-I-1D443></use></g><g data-mml-node=mo transform="translate(486,257) translate(-250 0)"><use data-c=5E xlink:href=#MJX-1773-TEX-N-5E></use></g></g></g><g data-mml-node=mo transform=translate(15219.5,0)><use data-c=2286 xlink:href=#MJX-1773-TEX-N-2286></use></g><g data-mml-node=mi transform=translate(16275.3,0)><use data-c=1D443 xlink:href=#MJX-1773-TEX-I-1D443></use></g><g data-mml-node=mo transform=translate(17026.3,0)><use data-c=2C xlink:href=#MJX-1773-TEX-N-2C></use></g><g data-mml-node=mo transform="translate(17470.9,0) translate(0 -0.5)"><use data-c=7C xlink:href=#MJX-1773-TEX-N-7C></use></g><g data-mml-node=TeXAtom data-mjx-texclass=ORD transform=translate(17748.9,0)><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D443 xlink:href=#MJX-1773-TEX-I-1D443></use></g><g data-mml-node=mo transform="translate(486,257) translate(-250 0)"><use data-c=5E xlink:href=#MJX-1773-TEX-N-5E></use></g></g></g><g data-mml-node=mo transform="translate(18499.9,0) translate(0 -0.5)"><use data-c=7C xlink:href=#MJX-1773-TEX-N-7C></use></g><g data-mml-node=mo transform=translate(19055.7,0)><use data-c=3D xlink:href=#MJX-1773-TEX-N-3D></use></g><g data-mml-node=mi transform=translate(20111.5,0)><use data-c=1D460 xlink:href=#MJX-1773-TEX-I-1D460></use></g></g></g></g></svg><svg data-labels=true preserveAspectRatio=xMaxYMid viewBox="1278 -1739.4 1 2978.9"><g data-labels=true transform="matrix(1 0 0 -1 0 0)"><g data-mml-node=mtd id=mjx-eqn:1 transform=translate(0,756.5)><text data-id-align=true></text><g data-idbox=true transform=translate(0,-750)><g data-mml-node=mtext><use data-c=28 xlink:href=#MJX-1773-TEX-N-28></use><use data-c=31 xlink:href=#MJX-1773-TEX-N-31 transform=translate(389,0)></use><use data-c=29 xlink:href=#MJX-1773-TEX-N-29 transform=translate(889,0)></use></g></g></g></g></svg></g></g></g></g></svg><mjx-assistive-mml unselectable=on display=block><math xmlns=http://www.w3.org/1998/Math/MathML display=block><mtable displaystyle=true><mlabeledtr><mtd><mtext>(1)</mtext></mtd><mtd><munderover><mo data-mjx-texclass=OP>∏</mo><mrow data-mjx-texclass=ORD><mi>i</mi><mo>=</mo><mn>1</mn></mrow><mi>N</mi></munderover><mtext>lik</mtext><mo stretchy=false>(</mo><msub><mrow data-mjx-texclass=ORD><mover><mi>p</mi><mo stretchy=false>~</mo></mover></mrow><mi>i</mi></msub><mo>,</mo><msub><mi>d</mi><mi>i</mi></msub><mo stretchy=false>)</mo><mo>≥</mo><munderover><mo data-mjx-texclass=OP>∏</mo><mrow data-mjx-texclass=ORD><mi>i</mi><mo>=</mo><mn>1</mn></mrow><mi>N</mi></munderover><mtext>lik</mtext><mo stretchy=false>(</mo><msub><mrow data-mjx-texclass=ORD><mover><mi>p</mi><mo stretchy=false>^</mo></mover></mrow><mi>i</mi></msub><mo>,</mo><msub><mi>d</mi><mi>i</mi></msub><mo stretchy=false>)</mo><mo>,</mo><mstyle scriptlevel=0><mspace width=1em></mspace></mstyle><mi mathvariant=normal>∀</mi><mrow data-mjx-texclass=ORD><mover><mi>P</mi><mo stretchy=false>^</mo></mover></mrow><mo>⊆</mo><mi>P</mi><mo>,</mo><mo data-mjx-texclass=ORD stretchy=false>|</mo><mrow data-mjx-texclass=ORD><mover><mi>P</mi><mo stretchy=false>^</mo></mover></mrow><mo data-mjx-texclass=ORD stretchy=false>|</mo><mo>=</mo><mi>s</mi></mtd></mlabeledtr></mtable></math></mjx-assistive-mml></mjx-container></div></div><p><span>Using the logarithm, we can reformulate the optimization problem as</span><div contenteditable=false spellcheck=false class="mathjax-block md-end-block md-math-block md-rawblock" id=mathjax-n11 cid=n11 mdtype=math_block data-math-tag-before=1 data-math-tag-after=2 data-math-labels=[]><div class=md-math-container><mjx-container class=MathJax jax=SVG display=true width=full style=min-width:63.253ex;position:relative><svg xmlns=http://www.w3.org/2000/svg width=100% height=6.74ex role=img focusable=false xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-2.804ex;min-width:63.253ex><defs><path id=MJX-1774-TEX-LO-2211 d="M60 948Q63 950 665 950H1267L1325 815Q1384 677 1388 669H1348L1341 683Q1320 724 1285 761Q1235 809 1174 838T1033 881T882 898T699 902H574H543H251L259 891Q722 258 724 252Q725 250 724 246Q721 243 460 -56L196 -356Q196 -357 407 -357Q459 -357 548 -357T676 -358Q812 -358 896 -353T1063 -332T1204 -283T1307 -196Q1328 -170 1348 -124H1388Q1388 -125 1381 -145T1356 -210T1325 -294L1267 -449L666 -450Q64 -450 61 -448Q55 -446 55 -439Q55 -437 57 -433L590 177Q590 178 557 222T452 366T322 544L56 909L55 924Q55 945 60 948Z"></path><path id=MJX-1774-TEX-I-1D456 d="M184 600Q184 624 203 642T247 661Q265 661 277 649T290 619Q290 596 270 577T226 557Q211 557 198 567T184 600ZM21 287Q21 295 30 318T54 369T98 420T158 442Q197 442 223 419T250 357Q250 340 236 301T196 196T154 83Q149 61 149 51Q149 26 166 26Q175 26 185 29T208 43T235 78T260 137Q263 149 265 151T282 153Q302 153 302 143Q302 135 293 112T268 61T223 11T161 -11Q129 -11 102 10T74 74Q74 91 79 106T122 220Q160 321 166 341T173 380Q173 404 156 404H154Q124 404 99 371T61 287Q60 286 59 284T58 281T56 279T53 278T49 278T41 278H27Q21 284 21 287Z"></path><path id=MJX-1774-TEX-N-3D d="M56 347Q56 360 70 367H707Q722 359 722 347Q722 336 708 328L390 327H72Q56 332 56 347ZM56 153Q56 168 72 173H708Q722 163 722 153Q722 140 707 133H70Q56 140 56 153Z"></path><path id=MJX-1774-TEX-N-31 d="M213 578L200 573Q186 568 160 563T102 556H83V602H102Q149 604 189 617T245 641T273 663Q275 666 285 666Q294 666 302 660V361L303 61Q310 54 315 52T339 48T401 46H427V0H416Q395 3 257 3Q121 3 100 0H88V46H114Q136 46 152 46T177 47T193 50T201 52T207 57T213 61V578Z"></path><path id=MJX-1774-TEX-I-1D441 d="M234 637Q231 637 226 637Q201 637 196 638T191 649Q191 676 202 682Q204 683 299 683Q376 683 387 683T401 677Q612 181 616 168L670 381Q723 592 723 606Q723 633 659 637Q635 637 635 648Q635 650 637 660Q641 676 643 679T653 683Q656 683 684 682T767 680Q817 680 843 681T873 682Q888 682 888 672Q888 650 880 642Q878 637 858 637Q787 633 769 597L620 7Q618 0 599 0Q585 0 582 2Q579 5 453 305L326 604L261 344Q196 88 196 79Q201 46 268 46H278Q284 41 284 38T282 19Q278 6 272 0H259Q228 2 151 2Q123 2 100 2T63 2T46 1Q31 1 31 10Q31 14 34 26T39 40Q41 46 62 46Q130 49 150 85Q154 91 221 362L289 634Q287 635 234 637Z"></path><path id=MJX-1774-TEX-N-6C d="M42 46H56Q95 46 103 60V68Q103 77 103 91T103 124T104 167T104 217T104 272T104 329Q104 366 104 407T104 482T104 542T103 586T103 603Q100 622 89 628T44 637H26V660Q26 683 28 683L38 684Q48 685 67 686T104 688Q121 689 141 690T171 693T182 694H185V379Q185 62 186 60Q190 52 198 49Q219 46 247 46H263V0H255L232 1Q209 2 183 2T145 3T107 3T57 1L34 0H26V46H42Z"></path><path id=MJX-1774-TEX-N-6F d="M28 214Q28 309 93 378T250 448Q340 448 405 380T471 215Q471 120 407 55T250 -10Q153 -10 91 57T28 214ZM250 30Q372 30 372 193V225V250Q372 272 371 288T364 326T348 362T317 390T268 410Q263 411 252 411Q222 411 195 399Q152 377 139 338T126 246V226Q126 130 145 91Q177 30 250 30Z"></path><path id=MJX-1774-TEX-N-67 d="M329 409Q373 453 429 453Q459 453 472 434T485 396Q485 382 476 371T449 360Q416 360 412 390Q410 404 415 411Q415 412 416 414V415Q388 412 363 393Q355 388 355 386Q355 385 359 381T368 369T379 351T388 325T392 292Q392 230 343 187T222 143Q172 143 123 171Q112 153 112 133Q112 98 138 81Q147 75 155 75T227 73Q311 72 335 67Q396 58 431 26Q470 -13 470 -72Q470 -139 392 -175Q332 -206 250 -206Q167 -206 107 -175Q29 -140 29 -75Q29 -39 50 -15T92 18L103 24Q67 55 67 108Q67 155 96 193Q52 237 52 292Q52 355 102 398T223 442Q274 442 318 416L329 409ZM299 343Q294 371 273 387T221 404Q192 404 171 388T145 343Q142 326 142 292Q142 248 149 227T179 192Q196 182 222 182Q244 182 260 189T283 207T294 227T299 242Q302 258 302 292T299 343ZM403 -75Q403 -50 389 -34T348 -11T299 -2T245 0H218Q151 0 138 -6Q118 -15 107 -34T95 -74Q95 -84 101 -97T122 -127T170 -155T250 -167Q319 -167 361 -139T403 -75Z"></path><path id=MJX-1774-TEX-N-2061 d></path><path id=MJX-1774-TEX-N-69 d="M69 609Q69 637 87 653T131 669Q154 667 171 652T188 609Q188 579 171 564T129 549Q104 549 87 564T69 609ZM247 0Q232 3 143 3Q132 3 106 3T56 1L34 0H26V46H42Q70 46 91 49Q100 53 102 60T104 102V205V293Q104 345 102 359T88 378Q74 385 41 385H30V408Q30 431 32 431L42 432Q52 433 70 434T106 436Q123 437 142 438T171 441T182 442H185V62Q190 52 197 50T232 46H255V0H247Z"></path><path id=MJX-1774-TEX-N-6B d="M36 46H50Q89 46 97 60V68Q97 77 97 91T97 124T98 167T98 217T98 272T98 329Q98 366 98 407T98 482T98 542T97 586T97 603Q94 622 83 628T38 637H20V660Q20 683 22 683L32 684Q42 685 61 686T98 688Q115 689 135 690T165 693T176 694H179V463L180 233L240 287Q300 341 304 347Q310 356 310 364Q310 383 289 385H284V431H293Q308 428 412 428Q475 428 484 431H489V385H476Q407 380 360 341Q286 278 286 274Q286 273 349 181T420 79Q434 60 451 53T500 46H511V0H505Q496 3 418 3Q322 3 307 0H299V46H306Q330 48 330 65Q330 72 326 79Q323 84 276 153T228 222L176 176V120V84Q176 65 178 59T189 49Q210 46 238 46H254V0H246Q231 3 137 3T28 0H20V46H36Z"></path><path id=MJX-1774-TEX-N-28 d="M94 250Q94 319 104 381T127 488T164 576T202 643T244 695T277 729T302 750H315H319Q333 750 333 741Q333 738 316 720T275 667T226 581T184 443T167 250T184 58T225 -81T274 -167T316 -220T333 -241Q333 -250 318 -250H315H302L274 -226Q180 -141 137 -14T94 250Z"></path><path id=MJX-1774-TEX-I-1D45D d="M23 287Q24 290 25 295T30 317T40 348T55 381T75 411T101 433T134 442Q209 442 230 378L240 387Q302 442 358 442Q423 442 460 395T497 281Q497 173 421 82T249 -10Q227 -10 210 -4Q199 1 187 11T168 28L161 36Q160 35 139 -51T118 -138Q118 -144 126 -145T163 -148H188Q194 -155 194 -157T191 -175Q188 -187 185 -190T172 -194Q170 -194 161 -194T127 -193T65 -192Q-5 -192 -24 -194H-32Q-39 -187 -39 -183Q-37 -156 -26 -148H-6Q28 -147 33 -136Q36 -130 94 103T155 350Q156 355 156 364Q156 405 131 405Q109 405 94 377T71 316T59 280Q57 278 43 278H29Q23 284 23 287ZM178 102Q200 26 252 26Q282 26 310 49T356 107Q374 141 392 215T411 325V331Q411 405 350 405Q339 405 328 402T306 393T286 380T269 365T254 350T243 336T235 326L232 322Q232 321 229 308T218 264T204 212Q178 106 178 102Z"></path><path id=MJX-1774-TEX-N-7E d="M179 251Q164 251 151 245T131 234T111 215L97 227L83 238Q83 239 95 253T121 283T142 304Q165 318 187 318T253 300T320 282Q335 282 348 288T368 299T388 318L402 306L416 295Q375 236 344 222Q330 215 313 215Q292 215 248 233T179 251Z"></path><path id=MJX-1774-TEX-N-2C d="M78 35T78 60T94 103T137 121Q165 121 187 96T210 8Q210 -27 201 -60T180 -117T154 -158T130 -185T117 -194Q113 -194 104 -185T95 -172Q95 -168 106 -156T131 -126T157 -76T173 -3V9L172 8Q170 7 167 6T161 3T152 1T140 0Q113 0 96 17Z"></path><path id=MJX-1774-TEX-I-1D451 d="M366 683Q367 683 438 688T511 694Q523 694 523 686Q523 679 450 384T375 83T374 68Q374 26 402 26Q411 27 422 35Q443 55 463 131Q469 151 473 152Q475 153 483 153H487H491Q506 153 506 145Q506 140 503 129Q490 79 473 48T445 8T417 -8Q409 -10 393 -10Q359 -10 336 5T306 36L300 51Q299 52 296 50Q294 48 292 46Q233 -10 172 -10Q117 -10 75 30T33 157Q33 205 53 255T101 341Q148 398 195 420T280 442Q336 442 364 400Q369 394 369 396Q370 400 396 505T424 616Q424 629 417 632T378 637H357Q351 643 351 645T353 664Q358 683 366 683ZM352 326Q329 405 277 405Q242 405 210 374T160 293Q131 214 119 129Q119 126 119 118T118 106Q118 61 136 44T179 26Q233 26 290 98L298 109L352 326Z"></path><path id=MJX-1774-TEX-N-29 d="M60 749L64 750Q69 750 74 750H86L114 726Q208 641 251 514T294 250Q294 182 284 119T261 12T224 -76T186 -143T145 -194T113 -227T90 -246Q87 -249 86 -250H74Q66 -250 63 -250T58 -247T55 -238Q56 -237 66 -225Q221 -64 221 250T66 725Q56 737 55 738Q55 746 60 749Z"></path><path id=MJX-1774-TEX-N-2265 d="M83 616Q83 624 89 630T99 636Q107 636 253 568T543 431T687 361Q694 356 694 346T687 331Q685 329 395 192L107 56H101Q83 58 83 76Q83 77 83 79Q82 86 98 95Q117 105 248 167Q326 204 378 228L626 346L360 472Q291 505 200 548Q112 589 98 597T83 616ZM84 -118Q84 -108 99 -98H678Q694 -104 694 -118Q694 -130 679 -138H98Q84 -131 84 -118Z"></path><path id=MJX-1774-TEX-N-5E d="M112 560L249 694L257 686Q387 562 387 560L361 531Q359 532 303 581L250 627L195 580Q182 569 169 557T148 538L140 532Q138 530 125 546L112 560Z"></path><path id=MJX-1774-TEX-N-2200 d="M0 673Q0 684 7 689T20 694Q32 694 38 680T82 567L126 451H430L473 566Q483 593 494 622T512 668T519 685Q524 694 538 694Q556 692 556 674Q556 670 426 329T293 -15Q288 -22 278 -22T263 -15Q260 -11 131 328T0 673ZM414 410Q414 411 278 411T142 410L278 55L414 410Z"></path><path id=MJX-1774-TEX-I-1D443 d="M287 628Q287 635 230 637Q206 637 199 638T192 648Q192 649 194 659Q200 679 203 681T397 683Q587 682 600 680Q664 669 707 631T751 530Q751 453 685 389Q616 321 507 303Q500 302 402 301H307L277 182Q247 66 247 59Q247 55 248 54T255 50T272 48T305 46H336Q342 37 342 35Q342 19 335 5Q330 0 319 0Q316 0 282 1T182 2Q120 2 87 2T51 1Q33 1 33 11Q33 13 36 25Q40 41 44 43T67 46Q94 46 127 49Q141 52 146 61Q149 65 218 339T287 628ZM645 554Q645 567 643 575T634 597T609 619T560 635Q553 636 480 637Q463 637 445 637T416 636T404 636Q391 635 386 627Q384 621 367 550T332 412T314 344Q314 342 395 342H407H430Q542 342 590 392Q617 419 631 471T645 554Z"></path><path id=MJX-1774-TEX-N-2286 d="M84 346Q84 468 166 546T360 635Q361 635 370 635T395 635T430 636T475 636T524 636H679Q694 628 694 616Q694 607 681 597L522 596H470H441Q366 596 338 592T266 568Q244 557 224 542T179 500T139 433T124 346V341Q124 253 185 185Q244 121 328 103Q348 98 366 98T522 96H681Q694 86 694 76Q694 64 679 56H526Q510 56 480 56T434 55Q350 55 289 71T172 141Q84 223 84 346ZM104 -131T104 -118T118 -98H679Q694 -106 694 -118T679 -138H118Q104 -131 104 -118Z"></path><path id=MJX-1774-TEX-N-7C d="M139 -249H137Q125 -249 119 -235V251L120 737Q130 750 139 750Q152 750 159 735V-235Q151 -249 141 -249H139Z"></path><path id=MJX-1774-TEX-I-1D460 d="M131 289Q131 321 147 354T203 415T300 442Q362 442 390 415T419 355Q419 323 402 308T364 292Q351 292 340 300T328 326Q328 342 337 354T354 372T367 378Q368 378 368 379Q368 382 361 388T336 399T297 405Q249 405 227 379T204 326Q204 301 223 291T278 274T330 259Q396 230 396 163Q396 135 385 107T352 51T289 7T195 -10Q118 -10 86 19T53 87Q53 126 74 143T118 160Q133 160 146 151T160 120Q160 94 142 76T111 58Q109 57 108 57T107 55Q108 52 115 47T146 34T201 27Q237 27 263 38T301 66T318 97T323 122Q323 150 302 164T254 181T195 196T148 231Q131 256 131 289Z"></path><path id=MJX-1774-TEX-N-32 d="M109 429Q82 429 66 447T50 491Q50 562 103 614T235 666Q326 666 387 610T449 465Q449 422 429 383T381 315T301 241Q265 210 201 149L142 93L218 92Q375 92 385 97Q392 99 409 186V189H449V186Q448 183 436 95T421 3V0H50V19V31Q50 38 56 46T86 81Q115 113 136 137Q145 147 170 174T204 211T233 244T261 278T284 308T305 340T320 369T333 401T340 431T343 464Q343 527 309 573T212 619Q179 619 154 602T119 569T109 550Q109 549 114 549Q132 549 151 535T170 489Q170 464 154 447T109 429Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform="scale(0.019042,-0.019042) translate(0, -1739.4)"><g data-mml-node=math><g data-mml-node=mtable transform="translate(2078,0) translate(-2078,0)"><g transform="translate(0 1739.4) matrix(1 0 0 -1 0 0) scale(52.51)"><svg data-table=true preserveAspectRatio=xMidYMid viewBox="11900.9 -1739.4 1 2978.9"><g transform="matrix(1 0 0 -1 0 0)"><g data-mml-node=mlabeledtr transform=translate(0,6.5)><g data-mml-node=mtd><g data-mml-node=munderover><g data-mml-node=mo><use data-c=2211 xlink:href=#MJX-1774-TEX-LO-2211></use></g><g data-mml-node=TeXAtom transform="translate(148.2,-1087.9) scale(0.707)" data-mjx-texclass=ORD><g data-mml-node=mi><use data-c=1D456 xlink:href=#MJX-1774-TEX-I-1D456></use></g><g data-mml-node=mo transform=translate(345,0)><use data-c=3D xlink:href=#MJX-1774-TEX-N-3D></use></g><g data-mml-node=mn transform=translate(1123,0)><use data-c=31 xlink:href=#MJX-1774-TEX-N-31></use></g></g><g data-mml-node=mi transform="translate(408,1150) scale(0.707)"><use data-c=1D441 xlink:href=#MJX-1774-TEX-I-1D441></use></g></g><g data-mml-node=mi transform=translate(1610.7,0)><use data-c=6C xlink:href=#MJX-1774-TEX-N-6C></use><use data-c=6F xlink:href=#MJX-1774-TEX-N-6F transform=translate(278,0)></use><use data-c=67 xlink:href=#MJX-1774-TEX-N-67 transform=translate(778,0)></use></g><g data-mml-node=mo transform=translate(2888.7,0)><use data-c=2061 xlink:href=#MJX-1774-TEX-N-2061></use></g><g data-mml-node=mtext transform=translate(3055.3,0)><use data-c=6C xlink:href=#MJX-1774-TEX-N-6C></use><use data-c=69 xlink:href=#MJX-1774-TEX-N-69 transform=translate(278,0)></use><use data-c=6B xlink:href=#MJX-1774-TEX-N-6B transform=translate(556,0)></use></g><g data-mml-node=mo transform=translate(4139.3,0)><use data-c=28 xlink:href=#MJX-1774-TEX-N-28></use></g><g data-mml-node=msub transform=translate(4528.3,0)><g data-mml-node=TeXAtom data-mjx-texclass=ORD><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D45D xlink:href=#MJX-1774-TEX-I-1D45D></use></g><g data-mml-node=mo transform="translate(334.8,332) translate(-250 0)"><use data-c=7E xlink:href=#MJX-1774-TEX-N-7E></use></g></g></g><g data-mml-node=mi transform="translate(536,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1774-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(5358.3,0)><use data-c=2C xlink:href=#MJX-1774-TEX-N-2C></use></g><g data-mml-node=msub transform=translate(5803,0)><g data-mml-node=mi><use data-c=1D451 xlink:href=#MJX-1774-TEX-I-1D451></use></g><g data-mml-node=mi transform="translate(553,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1774-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(6649.9,0)><use data-c=29 xlink:href=#MJX-1774-TEX-N-29></use></g><g data-mml-node=mo transform=translate(7316.7,0)><use data-c=2265 xlink:href=#MJX-1774-TEX-N-2265></use></g><g data-mml-node=munderover transform=translate(8372.5,0)><g data-mml-node=mo><use data-c=2211 xlink:href=#MJX-1774-TEX-LO-2211></use></g><g data-mml-node=TeXAtom transform="translate(148.2,-1087.9) scale(0.707)" data-mjx-texclass=ORD><g data-mml-node=mi><use data-c=1D456 xlink:href=#MJX-1774-TEX-I-1D456></use></g><g data-mml-node=mo transform=translate(345,0)><use data-c=3D xlink:href=#MJX-1774-TEX-N-3D></use></g><g data-mml-node=mn transform=translate(1123,0)><use data-c=31 xlink:href=#MJX-1774-TEX-N-31></use></g></g><g data-mml-node=mi transform="translate(408,1150) scale(0.707)"><use data-c=1D441 xlink:href=#MJX-1774-TEX-I-1D441></use></g></g><g data-mml-node=mi transform=translate(9983.1,0)><use data-c=6C xlink:href=#MJX-1774-TEX-N-6C></use><use data-c=6F xlink:href=#MJX-1774-TEX-N-6F transform=translate(278,0)></use><use data-c=67 xlink:href=#MJX-1774-TEX-N-67 transform=translate(778,0)></use></g><g data-mml-node=mo transform=translate(11261.1,0)><use data-c=2061 xlink:href=#MJX-1774-TEX-N-2061></use></g><g data-mml-node=mtext transform=translate(11427.8,0)><use data-c=6C xlink:href=#MJX-1774-TEX-N-6C></use><use data-c=69 xlink:href=#MJX-1774-TEX-N-69 transform=translate(278,0)></use><use data-c=6B xlink:href=#MJX-1774-TEX-N-6B transform=translate(556,0)></use></g><g data-mml-node=mo transform=translate(12511.8,0)><use data-c=28 xlink:href=#MJX-1774-TEX-N-28></use></g><g data-mml-node=msub transform=translate(12900.8,0)><g data-mml-node=TeXAtom data-mjx-texclass=ORD><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D45D xlink:href=#MJX-1774-TEX-I-1D45D></use></g><g data-mml-node=mo transform="translate(334.8,16) translate(-250 0)"><use data-c=5E xlink:href=#MJX-1774-TEX-N-5E></use></g></g></g><g data-mml-node=mi transform="translate(536,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1774-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(13730.7,0)><use data-c=2C xlink:href=#MJX-1774-TEX-N-2C></use></g><g data-mml-node=msub transform=translate(14175.4,0)><g data-mml-node=mi><use data-c=1D451 xlink:href=#MJX-1774-TEX-I-1D451></use></g><g data-mml-node=mi transform="translate(553,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1774-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(15022.4,0)><use data-c=29 xlink:href=#MJX-1774-TEX-N-29></use></g><g data-mml-node=mo transform=translate(15411.4,0)><use data-c=2C xlink:href=#MJX-1774-TEX-N-2C></use></g><g data-mml-node=mstyle transform=translate(15689.4,0)><g data-mml-node=mspace></g></g><g data-mml-node=mi transform=translate(16856,0)><use data-c=2200 xlink:href=#MJX-1774-TEX-N-2200></use></g><g data-mml-node=TeXAtom data-mjx-texclass=ORD transform=translate(17412,0)><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D443 xlink:href=#MJX-1774-TEX-I-1D443></use></g><g data-mml-node=mo transform="translate(486,257) translate(-250 0)"><use data-c=5E xlink:href=#MJX-1774-TEX-N-5E></use></g></g></g><g data-mml-node=mo transform=translate(18440.8,0)><use data-c=2286 xlink:href=#MJX-1774-TEX-N-2286></use></g><g data-mml-node=mi transform=translate(19496.6,0)><use data-c=1D443 xlink:href=#MJX-1774-TEX-I-1D443></use></g><g data-mml-node=mo transform=translate(20247.6,0)><use data-c=2C xlink:href=#MJX-1774-TEX-N-2C></use></g><g data-mml-node=mo transform="translate(20692.3,0) translate(0 -0.5)"><use data-c=7C xlink:href=#MJX-1774-TEX-N-7C></use></g><g data-mml-node=TeXAtom data-mjx-texclass=ORD transform=translate(20970.3,0)><g data-mml-node=mover><g data-mml-node=mi><use data-c=1D443 xlink:href=#MJX-1774-TEX-I-1D443></use></g><g data-mml-node=mo transform="translate(486,257) translate(-250 0)"><use data-c=5E xlink:href=#MJX-1774-TEX-N-5E></use></g></g></g><g data-mml-node=mo transform="translate(21721.3,0) translate(0 -0.5)"><use data-c=7C xlink:href=#MJX-1774-TEX-N-7C></use></g><g data-mml-node=mo transform=translate(22277,0)><use data-c=3D xlink:href=#MJX-1774-TEX-N-3D></use></g><g data-mml-node=mi transform=translate(23332.8,0)><use data-c=1D460 xlink:href=#MJX-1774-TEX-I-1D460></use></g></g></g></g></svg><svg data-labels=true preserveAspectRatio=xMaxYMid viewBox="1278 -1739.4 1 2978.9"><g data-labels=true transform="matrix(1 0 0 -1 0 0)"><g data-mml-node=mtd id=mjx-eqn:2 transform=translate(0,756.5)><text data-id-align=true></text><g data-idbox=true transform=translate(0,-750)><g data-mml-node=mtext><use data-c=28 xlink:href=#MJX-1774-TEX-N-28></use><use data-c=32 xlink:href=#MJX-1774-TEX-N-32 transform=translate(389,0)></use><use data-c=29 xlink:href=#MJX-1774-TEX-N-29 transform=translate(889,0)></use></g></g></g></g></svg></g></g></g></g></svg><mjx-assistive-mml unselectable=on display=block><math xmlns=http://www.w3.org/1998/Math/MathML display=block><mtable displaystyle=true><mlabeledtr><mtd><mtext>(2)</mtext></mtd><mtd><munderover><mo data-mjx-texclass=OP>∑</mo><mrow data-mjx-texclass=ORD><mi>i</mi><mo>=</mo><mn>1</mn></mrow><mi>N</mi></munderover><mi>log</mi><mo data-mjx-texclass=NONE>⁡</mo><mtext>lik</mtext><mo stretchy=false>(</mo><msub><mrow data-mjx-texclass=ORD><mover><mi>p</mi><mo stretchy=false>~</mo></mover></mrow><mi>i</mi></msub><mo>,</mo><msub><mi>d</mi><mi>i</mi></msub><mo stretchy=false>)</mo><mo>≥</mo><munderover><mo data-mjx-texclass=OP>∑</mo><mrow data-mjx-texclass=ORD><mi>i</mi><mo>=</mo><mn>1</mn></mrow><mi>N</mi></munderover><mi>log</mi><mo data-mjx-texclass=NONE>⁡</mo><mtext>lik</mtext><mo stretchy=false>(</mo><msub><mrow data-mjx-texclass=ORD><mover><mi>p</mi><mo stretchy=false>^</mo></mover></mrow><mi>i</mi></msub><mo>,</mo><msub><mi>d</mi><mi>i</mi></msub><mo stretchy=false>)</mo><mo>,</mo><mstyle scriptlevel=0><mspace width=1em></mspace></mstyle><mi mathvariant=normal>∀</mi><mrow data-mjx-texclass=ORD><mover><mi>P</mi><mo stretchy=false>^</mo></mover></mrow><mo>⊆</mo><mi>P</mi><mo>,</mo><mo data-mjx-texclass=ORD stretchy=false>|</mo><mrow data-mjx-texclass=ORD><mover><mi>P</mi><mo stretchy=false>^</mo></mover></mrow><mo data-mjx-texclass=ORD stretchy=false>|</mo><mo>=</mo><mi>s</mi></mtd></mlabeledtr></mtable></math></mjx-assistive-mml></mjx-container></div></div><p><span>This problem can be formulated as an assignment problem, which has a solution in </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=6.606ex height=2.451ex role=img focusable=false viewBox="0 -833.2 2919.8 1083.2" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.566ex><defs><path id=MJX-1749-TEX-I-1D442 d="M740 435Q740 320 676 213T511 42T304 -22Q207 -22 138 35T51 201Q50 209 50 244Q50 346 98 438T227 601Q351 704 476 704Q514 704 524 703Q621 689 680 617T740 435ZM637 476Q637 565 591 615T476 665Q396 665 322 605Q242 542 200 428T157 216Q157 126 200 73T314 19Q404 19 485 98T608 313Q637 408 637 476Z"></path><path id=MJX-1749-TEX-N-28 d="M94 250Q94 319 104 381T127 488T164 576T202 643T244 695T277 729T302 750H315H319Q333 750 333 741Q333 738 316 720T275 667T226 581T184 443T167 250T184 58T225 -81T274 -167T316 -220T333 -241Q333 -250 318 -250H315H302L274 -226Q180 -141 137 -14T94 250Z"></path><path id=MJX-1749-TEX-I-1D441 d="M234 637Q231 637 226 637Q201 637 196 638T191 649Q191 676 202 682Q204 683 299 683Q376 683 387 683T401 677Q612 181 616 168L670 381Q723 592 723 606Q723 633 659 637Q635 637 635 648Q635 650 637 660Q641 676 643 679T653 683Q656 683 684 682T767 680Q817 680 843 681T873 682Q888 682 888 672Q888 650 880 642Q878 637 858 637Q787 633 769 597L620 7Q618 0 599 0Q585 0 582 2Q579 5 453 305L326 604L261 344Q196 88 196 79Q201 46 268 46H278Q284 41 284 38T282 19Q278 6 272 0H259Q228 2 151 2Q123 2 100 2T63 2T46 1Q31 1 31 10Q31 14 34 26T39 40Q41 46 62 46Q130 49 150 85Q154 91 221 362L289 634Q287 635 234 637Z"></path><path id=MJX-1749-TEX-N-33 d="M127 463Q100 463 85 480T69 524Q69 579 117 622T233 665Q268 665 277 664Q351 652 390 611T430 522Q430 470 396 421T302 350L299 348Q299 347 308 345T337 336T375 315Q457 262 457 175Q457 96 395 37T238 -22Q158 -22 100 21T42 130Q42 158 60 175T105 193Q133 193 151 175T169 130Q169 119 166 110T159 94T148 82T136 74T126 70T118 67L114 66Q165 21 238 21Q293 21 321 74Q338 107 338 175V195Q338 290 274 322Q259 328 213 329L171 330L168 332Q166 335 166 348Q166 366 174 366Q202 366 232 371Q266 376 294 413T322 525V533Q322 590 287 612Q265 626 240 626Q208 626 181 615T143 592T132 580H135Q138 579 143 578T153 573T165 566T175 555T183 540T186 520Q186 498 172 481T127 463Z"></path><path id=MJX-1749-TEX-N-29 d="M60 749L64 750Q69 750 74 750H86L114 726Q208 641 251 514T294 250Q294 182 284 119T261 12T224 -76T186 -143T145 -194T113 -227T90 -246Q87 -249 86 -250H74Q66 -250 63 -250T58 -247T55 -238Q56 -237 66 -225Q221 -64 221 250T66 725Q56 737 55 738Q55 746 60 749Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D442 xlink:href=#MJX-1749-TEX-I-1D442></use></g><g data-mml-node=mo transform=translate(763,0)><use data-c=28 xlink:href=#MJX-1749-TEX-N-28></use></g><g data-mml-node=msup transform=translate(1152,0)><g data-mml-node=mi><use data-c=1D441 xlink:href=#MJX-1749-TEX-I-1D441></use></g><g data-mml-node=mn transform="translate(975.3,363) scale(0.707)"><use data-c=33 xlink:href=#MJX-1749-TEX-N-33></use></g></g><g data-mml-node=mo transform=translate(2530.8,0)><use data-c=29 xlink:href=#MJX-1749-TEX-N-29></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>O</mi><mo stretchy=false>(</mo><msup><mi>N</mi><mn>3</mn></msup><mo stretchy=false>)</mo></math></mjx-assistive-mml></mjx-container><span> time complexity.</span><p><span>We define a bipartite graph where the left side consists of all points and the right side consists of distributions. The weight of the edge connecting point </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=0.781ex height=1.52ex role=img focusable=false viewBox="0 -661 345 672" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.025ex><defs><path id=MJX-1281-TEX-I-1D456 d="M184 600Q184 624 203 642T247 661Q265 661 277 649T290 619Q290 596 270 577T226 557Q211 557 198 567T184 600ZM21 287Q21 295 30 318T54 369T98 420T158 442Q197 442 223 419T250 357Q250 340 236 301T196 196T154 83Q149 61 149 51Q149 26 166 26Q175 26 185 29T208 43T235 78T260 137Q263 149 265 151T282 153Q302 153 302 143Q302 135 293 112T268 61T223 11T161 -11Q129 -11 102 10T74 74Q74 91 79 106T122 220Q160 321 166 341T173 380Q173 404 156 404H154Q124 404 99 371T61 287Q60 286 59 284T58 281T56 279T53 278T49 278T41 278H27Q21 284 21 287Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D456 xlink:href=#MJX-1281-TEX-I-1D456></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>i</mi></math></mjx-assistive-mml></mjx-container><span> and distribution </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=0.932ex height=1.957ex role=img focusable=false viewBox="0 -661 412 865" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.462ex><defs><path id=MJX-1279-TEX-I-1D457 d="M297 596Q297 627 318 644T361 661Q378 661 389 651T403 623Q403 595 384 576T340 557Q322 557 310 567T297 596ZM288 376Q288 405 262 405Q240 405 220 393T185 362T161 325T144 293L137 279Q135 278 121 278H107Q101 284 101 286T105 299Q126 348 164 391T252 441Q253 441 260 441T272 442Q296 441 316 432Q341 418 354 401T367 348V332L318 133Q267 -67 264 -75Q246 -125 194 -164T75 -204Q25 -204 7 -183T-12 -137Q-12 -110 7 -91T53 -71Q70 -71 82 -81T95 -112Q95 -148 63 -167Q69 -168 77 -168Q111 -168 139 -140T182 -74L193 -32Q204 11 219 72T251 197T278 308T289 365Q289 372 288 376Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=1D457 xlink:href=#MJX-1279-TEX-I-1D457></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>j</mi></math></mjx-assistive-mml></mjx-container><span> is the value </span><mjx-container class=MathJax jax=SVG style=position:relative><svg xmlns=http://www.w3.org/2000/svg width=12.388ex height=2.363ex role=img focusable=false viewBox="0 -750 5475.6 1044.2" xmlns:xlink=http://www.w3.org/1999/xlink aria-hidden=true style=vertical-align:-0.666ex><defs><path id=MJX-1769-TEX-N-6C d="M42 46H56Q95 46 103 60V68Q103 77 103 91T103 124T104 167T104 217T104 272T104 329Q104 366 104 407T104 482T104 542T103 586T103 603Q100 622 89 628T44 637H26V660Q26 683 28 683L38 684Q48 685 67 686T104 688Q121 689 141 690T171 693T182 694H185V379Q185 62 186 60Q190 52 198 49Q219 46 247 46H263V0H255L232 1Q209 2 183 2T145 3T107 3T57 1L34 0H26V46H42Z"></path><path id=MJX-1769-TEX-N-6F d="M28 214Q28 309 93 378T250 448Q340 448 405 380T471 215Q471 120 407 55T250 -10Q153 -10 91 57T28 214ZM250 30Q372 30 372 193V225V250Q372 272 371 288T364 326T348 362T317 390T268 410Q263 411 252 411Q222 411 195 399Q152 377 139 338T126 246V226Q126 130 145 91Q177 30 250 30Z"></path><path id=MJX-1769-TEX-N-67 d="M329 409Q373 453 429 453Q459 453 472 434T485 396Q485 382 476 371T449 360Q416 360 412 390Q410 404 415 411Q415 412 416 414V415Q388 412 363 393Q355 388 355 386Q355 385 359 381T368 369T379 351T388 325T392 292Q392 230 343 187T222 143Q172 143 123 171Q112 153 112 133Q112 98 138 81Q147 75 155 75T227 73Q311 72 335 67Q396 58 431 26Q470 -13 470 -72Q470 -139 392 -175Q332 -206 250 -206Q167 -206 107 -175Q29 -140 29 -75Q29 -39 50 -15T92 18L103 24Q67 55 67 108Q67 155 96 193Q52 237 52 292Q52 355 102 398T223 442Q274 442 318 416L329 409ZM299 343Q294 371 273 387T221 404Q192 404 171 388T145 343Q142 326 142 292Q142 248 149 227T179 192Q196 182 222 182Q244 182 260 189T283 207T294 227T299 242Q302 258 302 292T299 343ZM403 -75Q403 -50 389 -34T348 -11T299 -2T245 0H218Q151 0 138 -6Q118 -15 107 -34T95 -74Q95 -84 101 -97T122 -127T170 -155T250 -167Q319 -167 361 -139T403 -75Z"></path><path id=MJX-1769-TEX-N-2061 d></path><path id=MJX-1769-TEX-N-69 d="M69 609Q69 637 87 653T131 669Q154 667 171 652T188 609Q188 579 171 564T129 549Q104 549 87 564T69 609ZM247 0Q232 3 143 3Q132 3 106 3T56 1L34 0H26V46H42Q70 46 91 49Q100 53 102 60T104 102V205V293Q104 345 102 359T88 378Q74 385 41 385H30V408Q30 431 32 431L42 432Q52 433 70 434T106 436Q123 437 142 438T171 441T182 442H185V62Q190 52 197 50T232 46H255V0H247Z"></path><path id=MJX-1769-TEX-N-6B d="M36 46H50Q89 46 97 60V68Q97 77 97 91T97 124T98 167T98 217T98 272T98 329Q98 366 98 407T98 482T98 542T97 586T97 603Q94 622 83 628T38 637H20V660Q20 683 22 683L32 684Q42 685 61 686T98 688Q115 689 135 690T165 693T176 694H179V463L180 233L240 287Q300 341 304 347Q310 356 310 364Q310 383 289 385H284V431H293Q308 428 412 428Q475 428 484 431H489V385H476Q407 380 360 341Q286 278 286 274Q286 273 349 181T420 79Q434 60 451 53T500 46H511V0H505Q496 3 418 3Q322 3 307 0H299V46H306Q330 48 330 65Q330 72 326 79Q323 84 276 153T228 222L176 176V120V84Q176 65 178 59T189 49Q210 46 238 46H254V0H246Q231 3 137 3T28 0H20V46H36Z"></path><path id=MJX-1769-TEX-N-28 d="M94 250Q94 319 104 381T127 488T164 576T202 643T244 695T277 729T302 750H315H319Q333 750 333 741Q333 738 316 720T275 667T226 581T184 443T167 250T184 58T225 -81T274 -167T316 -220T333 -241Q333 -250 318 -250H315H302L274 -226Q180 -141 137 -14T94 250Z"></path><path id=MJX-1769-TEX-I-1D45D d="M23 287Q24 290 25 295T30 317T40 348T55 381T75 411T101 433T134 442Q209 442 230 378L240 387Q302 442 358 442Q423 442 460 395T497 281Q497 173 421 82T249 -10Q227 -10 210 -4Q199 1 187 11T168 28L161 36Q160 35 139 -51T118 -138Q118 -144 126 -145T163 -148H188Q194 -155 194 -157T191 -175Q188 -187 185 -190T172 -194Q170 -194 161 -194T127 -193T65 -192Q-5 -192 -24 -194H-32Q-39 -187 -39 -183Q-37 -156 -26 -148H-6Q28 -147 33 -136Q36 -130 94 103T155 350Q156 355 156 364Q156 405 131 405Q109 405 94 377T71 316T59 280Q57 278 43 278H29Q23 284 23 287ZM178 102Q200 26 252 26Q282 26 310 49T356 107Q374 141 392 215T411 325V331Q411 405 350 405Q339 405 328 402T306 393T286 380T269 365T254 350T243 336T235 326L232 322Q232 321 229 308T218 264T204 212Q178 106 178 102Z"></path><path id=MJX-1769-TEX-I-1D456 d="M184 600Q184 624 203 642T247 661Q265 661 277 649T290 619Q290 596 270 577T226 557Q211 557 198 567T184 600ZM21 287Q21 295 30 318T54 369T98 420T158 442Q197 442 223 419T250 357Q250 340 236 301T196 196T154 83Q149 61 149 51Q149 26 166 26Q175 26 185 29T208 43T235 78T260 137Q263 149 265 151T282 153Q302 153 302 143Q302 135 293 112T268 61T223 11T161 -11Q129 -11 102 10T74 74Q74 91 79 106T122 220Q160 321 166 341T173 380Q173 404 156 404H154Q124 404 99 371T61 287Q60 286 59 284T58 281T56 279T53 278T49 278T41 278H27Q21 284 21 287Z"></path><path id=MJX-1769-TEX-N-2C d="M78 35T78 60T94 103T137 121Q165 121 187 96T210 8Q210 -27 201 -60T180 -117T154 -158T130 -185T117 -194Q113 -194 104 -185T95 -172Q95 -168 106 -156T131 -126T157 -76T173 -3V9L172 8Q170 7 167 6T161 3T152 1T140 0Q113 0 96 17Z"></path><path id=MJX-1769-TEX-I-1D451 d="M366 683Q367 683 438 688T511 694Q523 694 523 686Q523 679 450 384T375 83T374 68Q374 26 402 26Q411 27 422 35Q443 55 463 131Q469 151 473 152Q475 153 483 153H487H491Q506 153 506 145Q506 140 503 129Q490 79 473 48T445 8T417 -8Q409 -10 393 -10Q359 -10 336 5T306 36L300 51Q299 52 296 50Q294 48 292 46Q233 -10 172 -10Q117 -10 75 30T33 157Q33 205 53 255T101 341Q148 398 195 420T280 442Q336 442 364 400Q369 394 369 396Q370 400 396 505T424 616Q424 629 417 632T378 637H357Q351 643 351 645T353 664Q358 683 366 683ZM352 326Q329 405 277 405Q242 405 210 374T160 293Q131 214 119 129Q119 126 119 118T118 106Q118 61 136 44T179 26Q233 26 290 98L298 109L352 326Z"></path><path id=MJX-1769-TEX-I-1D457 d="M297 596Q297 627 318 644T361 661Q378 661 389 651T403 623Q403 595 384 576T340 557Q322 557 310 567T297 596ZM288 376Q288 405 262 405Q240 405 220 393T185 362T161 325T144 293L137 279Q135 278 121 278H107Q101 284 101 286T105 299Q126 348 164 391T252 441Q253 441 260 441T272 442Q296 441 316 432Q341 418 354 401T367 348V332L318 133Q267 -67 264 -75Q246 -125 194 -164T75 -204Q25 -204 7 -183T-12 -137Q-12 -110 7 -91T53 -71Q70 -71 82 -81T95 -112Q95 -148 63 -167Q69 -168 77 -168Q111 -168 139 -140T182 -74L193 -32Q204 11 219 72T251 197T278 308T289 365Q289 372 288 376Z"></path><path id=MJX-1769-TEX-N-29 d="M60 749L64 750Q69 750 74 750H86L114 726Q208 641 251 514T294 250Q294 182 284 119T261 12T224 -76T186 -143T145 -194T113 -227T90 -246Q87 -249 86 -250H74Q66 -250 63 -250T58 -247T55 -238Q56 -237 66 -225Q221 -64 221 250T66 725Q56 737 55 738Q55 746 60 749Z"></path></defs><g stroke=currentColor fill=currentColor stroke-width=0 transform=scale(1,-1)><g data-mml-node=math><g data-mml-node=mi><use data-c=6C xlink:href=#MJX-1769-TEX-N-6C></use><use data-c=6F xlink:href=#MJX-1769-TEX-N-6F transform=translate(278,0)></use><use data-c=67 xlink:href=#MJX-1769-TEX-N-67 transform=translate(778,0)></use></g><g data-mml-node=mo transform=translate(1278,0)><use data-c=2061 xlink:href=#MJX-1769-TEX-N-2061></use></g><g data-mml-node=mtext transform=translate(1444.7,0)><use data-c=6C xlink:href=#MJX-1769-TEX-N-6C></use><use data-c=69 xlink:href=#MJX-1769-TEX-N-69 transform=translate(278,0)></use><use data-c=6B xlink:href=#MJX-1769-TEX-N-6B transform=translate(556,0)></use></g><g data-mml-node=mo transform=translate(2528.7,0)><use data-c=28 xlink:href=#MJX-1769-TEX-N-28></use></g><g data-mml-node=msub transform=translate(2917.7,0)><g data-mml-node=mi><use data-c=1D45D xlink:href=#MJX-1769-TEX-I-1D45D></use></g><g data-mml-node=mi transform="translate(536,-150) scale(0.707)"><use data-c=1D456 xlink:href=#MJX-1769-TEX-I-1D456></use></g></g><g data-mml-node=mo transform=translate(3747.6,0)><use data-c=2C xlink:href=#MJX-1769-TEX-N-2C></use></g><g data-mml-node=msub transform=translate(4192.3,0)><g data-mml-node=mi><use data-c=1D451 xlink:href=#MJX-1769-TEX-I-1D451></use></g><g data-mml-node=mi transform="translate(553,-150) scale(0.707)"><use data-c=1D457 xlink:href=#MJX-1769-TEX-I-1D457></use></g></g><g data-mml-node=mo transform=translate(5086.6,0)><use data-c=29 xlink:href=#MJX-1769-TEX-N-29></use></g></g></g></svg><mjx-assistive-mml unselectable=on display=inline><math xmlns=http://www.w3.org/1998/Math/MathML><mi>log</mi><mo data-mjx-texclass=NONE>⁡</mo><mtext>lik</mtext><mo stretchy=false>(</mo><msub><mi>p</mi><mi>i</mi></msub><mo>,</mo><msub><mi>d</mi><mi>j</mi></msub><mo stretchy=false>)</mo></math></mjx-assistive-mml></mjx-container><span>.</span><p><img src="..." width=350></p></div></div>
"""

### Code

In [ ]:
mean = [0, 0]
cov = [[1, 0], [0, 100]]
pts = np.random.multivariate_normal(mean, cov, size=800)

visualize_points(pts).show()

In [ ]:
# each row has the values for mutlivariate normal distributions
distributions = np.array([
    [[0, 0], [10, 0], [0, 10]]
])

plot_confidence_ellipses(distributions)[0].show()

In [ ]:
import numpy as np

def mahalanobis_distance(point, mean, cov):
    diff = point - mean
    cov_inv = np.linalg.inv(cov)
    distance = np.sqrt(np.dot(np.dot(diff.T, cov_inv), diff))
    return distance

def likelihood_on_ellipse_edge(mean, cov, n_std):
    # In 2D, the normalization factor for the PDF of the multivariate normal distribution
    norm_factor = 1 / (2 * np.pi * np.sqrt(np.linalg.det(cov)))

    # Compute the Mahalanobis distance (which is n_std on the edge of the ellipse)
    mahalanobis_dist = n_std

    # The exponent term in the PDF (remember that (x-mean)^T * cov_inv * (x-mean) = (mahalanobis_dist)^2)
    exponent_term = -0.5 * (mahalanobis_dist**2)

    # Full likelihood computation
    likelihood = norm_factor * np.exp(exponent_term)
    return likelihood

# Example usage
mean = np.array([0, 0])
cov = np.array([[1, 0.5], [0.5, 1]])
n_std = 1.96  # 95% confidence interval

likelihood = likelihood_on_ellipse_edge(mean, cov, n_std)
print("Likelihood on the edge of the ellipse:", likelihood)

In [ ]:
# each row has the values for mutlivariate normal distributions
distributions = np.array([
    [[3, 0], [1, 0], [0, 1]],
    [[2, 0], [1, 0.5], [0.5, 1]],
    [[0, 2], [1, -0.2], [-0.2, 1]],
    [[2, 1], [1, 0.5], [0.5, 1]],
    [[0, 2], [1, -1.2], [-0.2, 1]],
    [[-2, -2], [1, 0.3], [0.3, 1]]
])

plot_confidence_ellipses(distributions)[0].show()

### Points to Distribution

In [ ]:
display(HTML(assignment_problem_html))

In [ ]:
from scipy.stats import multivariate_normal

def calc_likelihoods_for_distributions_and_points(matrix_dist, matrix_points):
    if len(matrix_dist.shape) == 2:
        means = [matrix_dist[0]]
        covariances = [matrix_dist[1:]]
    else:
        means = matrix_dist[:, 0]
        covariances = matrix_dist[:, 1:]
    likelihoods = np.array([multivariate_normal(mean=means[i], cov=covariances[i]).pdf(points) for i in range(len(means))])
    
    return likelihoods.T 

In [ ]:
def calc_point_indices_to_distributions(distributions, points, treshhold_likelihood = 0):
    if points.shape[0] == 0:
        return np.zeros(distributions.shape[0]) - 1
    
    if points.shape[0] == 1 and distributions.shape[0] == 1:
        likelihood_entries = calc_likelihoods_for_distributions_and_points(distributions, points)
    
        if likelihood_entries < treshhold_likelihood: return np.array([-1])
        return np.array([0])

    likelihood_entries, likelihood_indicies_to_filter = calc_likelihood_entries_to_distributions(distributions, points, treshhold_likelihood)

    return calc_point_indicies_to_likelihoods(likelihood_entries, likelihood_indicies_to_filter)


def calc_point_indicies_to_likelihoods(likelihood_entries, likelihood_indicies_to_filter):
    inv_percentage_of_likelihood = np.log(likelihood_entries)/ np.sum(np.log(likelihood_entries), axis=0)

    cost_matrix = np.log(inv_percentage_of_likelihood)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    max_index = max(col_ind) + 1
    points_to_distributions = np.zeros(max_index, dtype=row_ind.dtype) - 1
    
    points_to_distributions[col_ind] = row_ind

    # if punished points still come up ahead, set the association to -1
    for point_index, distribution_index in enumerate(col_ind):
        if likelihood_indicies_to_filter[point_index, distribution_index]:
            points_to_distributions[point_index] = -1
    
    return points_to_distributions

def calc_likelihood_entries_to_distributions(distributions, points, treshhold_likelihood=0):
    likelihoods = calc_likelihoods_for_distributions_and_points(distributions, points)
    sorted_indices = np.argsort(likelihoods, axis=0)
    ranks = np.zeros_like(likelihoods, dtype=int)
    n_rows, n_cols = likelihoods.shape
    ranks[sorted_indices, np.arange(n_cols)] = np.tile(np.arange(n_rows), (n_cols, 1)).T
    
    mask_binary = np.array((n_rows - ranks) <= n_cols, dtype=int)
    cumsum_array = np.cumsum(mask_binary, axis=0)
    
    s = min(n_rows, n_cols)
    extended_likelihood_entries = np.zeros((s+1, n_cols + 1))
    
    mask  = np.array((n_rows - ranks) <= n_cols)
    
    points_to_consider = np.where(np.any(mask, axis=1))[0]
    filtered_points = points[points_to_consider]
    
    likelihood_entries = calc_likelihoods_for_distributions_and_points(distributions, filtered_points)
    likelihood_indicies_to_filter = likelihood_entries < treshhold_likelihood
    # punish points that should be filtered out
    likelihood_entries[likelihood_indicies_to_filter] = 1e-50

    return likelihood_entries, likelihood_indicies_to_filter

In [ ]:
def plot_distributions_with_colored_points(distributions, points, points_to_distribution, n_std=1.96):
    fig, ax = plot_confidence_ellipses(distributions, n_std)
    
    # Plot all points in a default color (black)
    ax.plot(points[:, 0], points[:, 1], 'ko', label='All Points')

    # Use the same colormap to get consistent colors
    cmap = plt.get_cmap('viridis')
    colors = cmap(np.linspace(0, 1, len(distributions)))

    for dist_idx, point_idx in enumerate(points_to_distribution):
        if point_idx == -1: continue
        ax.plot(points[point_idx, 0], points[point_idx, 1], 'o', color=colors[dist_idx], label=f'Point to Distribution {dist_idx}')
    
    ax.legend()
    return plt, ax

In [ ]:
# # each row has the values for mutlivariate normal distributions
# distributions = np.array([
#     [[3, 0], [1, 0], [0, 1]]
# ])

# plot_confidence_ellipses(distributions)[0].show()

In [ ]:
points = np.random.uniform(-8, 8, (30, 2))
points_to_distributions = calc_point_indices_to_distributions(distributions, points, 0)
plot_distributions_with_colored_points(distributions, points, points_to_distributions, n_std=1.96)[0].show()

In [ ]:
# points, points_to_distributions

#### Gaussian Mixture Models

The idea is, that we want to have probability distributions from different iterations. They will be weighted and observed as a mixture model. Based on that we will do the predictions as always

In [ ]:
def calc_point_indices_to_distributions(distributions, points, treshhold_likelihood = 0):
    if points.shape[0] == 0:
        return np.zeros(distributions.shape[0]) - 1
    
    if points.shape[0] == 1 and distributions.shape[0] == 1:
        likelihood_entries = calc_likelihoods_for_distributions_and_points(distributions, points)
    
        if likelihood_entries < treshhold_likelihood: return np.array([-1])
        return np.array([0])

    likelihood_entries, likelihood_indicies_to_filter = calc_likelihood_entries_to_distributions(distributions, points, treshhold_likelihood)

    return calc_point_indicies_to_likelihoods(likelihood_entries, likelihood_indicies_to_filter)

def calc_point_indicies_to_likelihoods_no_filter(likelihood_entries):
    cost_matrix = -1 * np.log(likelihood_entries)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    # max_index = max(col_ind) + 1
    points_to_distributions = np.zeros(likelihood_entries.shape[1], dtype=row_ind.dtype) - 1
    points_to_distributions[col_ind] = row_ind

    return points_to_distributions

def calc_point_indicies_to_likelihoods(likelihood_entries, likelihood_indicies_to_filter):
    # inv_percentage_of_likelihood = np.log(likelihood_entries)/ np.sum(np.log(likelihood_entries), axis=0)

    cost_matrix = -1 * np.log(likelihood_entries)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    
    # max_index = max(col_ind) + 1
    # points_to_distributions = np.zeros(max_index, dtype=row_ind.dtype) - 1
    points_to_distributions = np.zeros(likelihood_entries.shape[1], dtype=row_ind.dtype) - 1
    
    points_to_distributions[col_ind] = row_ind

    # if punished points still come up ahead, set the association to -1
    for point_index, distribution_index in enumerate(col_ind):
        if likelihood_indicies_to_filter[point_index, distribution_index]:
            points_to_distributions[point_index] = -1
    
    return points_to_distributions

def calc_likelihood_entries_to_distributions(distributions, points, treshhold_likelihood=0):
    likelihoods = calc_likelihoods_for_distributions_and_points(distributions, points)
    sorted_indices = np.argsort(likelihoods, axis=0)
    ranks = np.zeros_like(likelihoods, dtype=int)
    n_rows, n_cols = likelihoods.shape
    ranks[sorted_indices, np.arange(n_cols)] = np.tile(np.arange(n_rows), (n_cols, 1)).T
    
    mask_binary = np.array((n_rows - ranks) <= n_cols, dtype=int)
    cumsum_array = np.cumsum(mask_binary, axis=0)
    
    s = min(n_rows, n_cols)
    extended_likelihood_entries = np.zeros((s+1, n_cols + 1))
    
    mask  = np.array((n_rows - ranks) <= n_cols)
    
    points_to_consider = np.where(np.any(mask, axis=1))[0]
    filtered_points = points[points_to_consider]
    
    likelihood_entries = calc_likelihoods_for_distributions_and_points(distributions, filtered_points)
    likelihood_indicies_to_filter = likelihood_entries < treshhold_likelihood
    # punish points that should be filtered out
    likelihood_entries[likelihood_indicies_to_filter] = 1e-50

    return likelihood_entries, likelihood_indicies_to_filter

In [ ]:
def calculate_matrix_order(matrix):
    return np.argsort(input_matrix, axis=1)

In [ ]:
def reorder_matrix_with_matrix(input_matrix, order_matrix):
    return np.array([input_matrix[i][matrix_order[i]] for i in range(input_matrix.shape[0])])

In [ ]:
def calc_point_indices_to_gmm(distributions, ids, weights, points, treshhold_likelihood=0):
    unique_ids = np.unique(ids)
    id_to_order = {id_val: index for index, id_val in enumerate(unique_ids)}
    id_ordering = np.vectorize(id_to_order.get)(ids)
    
    likelihoods_time_distributions_to_points = np.zeros((len(unique_ids), points.shape[0]))
    
    for i, iteration_distributions in enumerate(distributions):
        likelihood_entries, likelihood_indicies_to_filter = calc_likelihood_entries_to_distributions(iteration_distributions, points)
        likelihood_entries[likelihood_indicies_to_filter] = 0
        likelihoods_time_distributions_to_points[id_ordering[0, :]] += weights[i] * likelihood_entries.T
    
    likelihoods_time_distributions_to_points[likelihoods_time_distributions_to_points == 0] = 1e-50
    point_mappings = calc_point_indicies_to_likelihoods_no_filter(likelihoods_time_distributions_to_points)
    for i, point_mapping in enumerate(point_mappings):
        if(point_mapping == -1): continue
        point_mappings[i] = unique_ids[point_mappings[i]]
    
    return point_mappings

In [ ]:
points = np.random.uniform(-5, 13, (10, 2))

distributions_a = np.array([
    [[3, 4], [1, 0], [0, 1]],
    # [[2, 0], [1, 0], [0, 1]]
])

distributions_b = np.array([
    [[4, 4], [1, 0], [0, 1]],
    [[3, 0], [1, 0], [0, 1]]
])

distributions_c = np.array([
    [[4, 0], [1, 0], [0, 1]],
    [[5, 4], [1, 0], [0, 1]]
])

distributions = [distributions_a, distributions_b, distributions_c]
# distributions = [distributions_a]

weights = [0.7, 0.2, 0.1]

ids = [
    [0, 1],
    [0, 1],
    [1, 0]
]

point_mapping_to_ids = calc_point_indices_to_gmm(distributions, ids, weights, points)
plt = plot_gmm_distributions_with_points(distributions, weights, ids, points, point_mapping_to_ids)
plt.show()

In [ ]:
points, point_mapping_to_ids

### Gaussian Processes